In [9]:
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
import pandas as pd

In [2]:
afq_dataset = AFQDataset.from_files(
    fn_nodes="../data/raw/combined_tract_profiles.csv",
    fn_subjects="../data/raw/participants_updated_id.csv",
    dwi_metrics=["dki_fa", "dki_md", "dki_mk"],
    index_col="subject_id",
    target_cols=["age", "dl_qc_score", "scan_site_id"],
    label_encode_cols=["scan_site_id"]
)

In [3]:
afq_dataset.drop_target_na()

In [4]:
print(len(afq_dataset.subjects))
print(afq_dataset.X.shape)
print(afq_dataset.y.shape)

1865
(1865, 7200)
(1865, 3)


In [5]:
full_dataset = list(afq_dataset.as_tensorflow_dataset().as_numpy_iterator())

2022-04-07 10:25:49.217761: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
X = np.concatenate([xx[0][None] for xx in full_dataset], 0)
y = np.array([yy[1][0] for yy in full_dataset])
qc = np.array([yy[1][1] for yy in full_dataset])
site = np.array([yy[1][2] for yy in full_dataset])

In [7]:
X = X[qc>0]
y = y[qc>0]
site = site[qc>0]

In [8]:
# # Split the data by sites
# X_0 = X[site==0]
# y_0 = y[site==0]
# X_3 = X[site==3]
# y_3 = y[site==3]
# X_4 = X[site==4]
# y_4 = y[site==4]

In [9]:
# # Split the data into train and test sets:
# X_0_train, X_0_test, y_0_train, y_0_test = train_test_split(X_0, y_0, test_size=0.2, random_state=42)
# X_3_train, X_3_test, y_3_train, y_3_test = train_test_split(X_3, y_3, test_size=0.2, random_state=42)
# X_4_train, X_4_test, y_4_train, y_4_test = train_test_split(X_4, y_4, test_size=0.2, random_state=42)

In [10]:
# imputer = SimpleImputer(strategy="median")

In [11]:
# # Impute train and test separately:
# X_0_train = np.concatenate([imputer.fit_transform(X_0_train[..., ii])[:, :, None] for ii in range(X_0_train.shape[-1])], -1)
# X_0_test = np.concatenate([imputer.fit_transform(X_0_test[..., ii])[:, :, None] for ii in range(X_0_test.shape[-1])], -1)
# X_3_train = np.concatenate([imputer.fit_transform(X_3_train[..., ii])[:, :, None] for ii in range(X_3_train.shape[-1])], -1)
# X_3_test = np.concatenate([imputer.fit_transform(X_3_test[..., ii])[:, :, None] for ii in range(X_3_test.shape[-1])], -1)
# X_4_train = np.concatenate([imputer.fit_transform(X_4_train[..., ii])[:, :, None] for ii in range(X_4_train.shape[-1])], -1)
# X_4_test = np.concatenate([imputer.fit_transform(X_4_test[..., ii])[:, :, None] for ii in range(X_4_test.shape[-1])], -1)

In [8]:
n_epochs = 1000

# EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    mode="min",
    patience=100
)

# ReduceLROnPlateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=20,
    verbose=1,
)

In [30]:
# Generate evaluation results and correlation coeffcients combined in a dataframe, and history
def single_cross_site(model_name, name_str, ckpt_filepath, lr,
                      site_1, site_2, site_3, X, y):
    # Split the data by sites
    X_1 = X[site==site_1]
    y_1 = y[site==site_1]
    X_2 = X[site==site_2]
    y_2 = y[site==site_2]
    X_3 = X[site==site_3]
    y_3 = y[site==site_3]
    # Split the data into train and test sets:
    X_train, X_test1, y_train, y_test1 = train_test_split(X_1, y_1, test_size=0.2)
    _, X_test2, _, y_test2 = train_test_split(X_2, y_2, test_size=0.2)
    _, X_test3, _, y_test3 = train_test_split(X_3, y_3, test_size=0.2)
    imputer = SimpleImputer(strategy="median")
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test1 = np.concatenate([imputer.fit_transform(X_test1[..., ii])[:, :, None] for ii in range(X_test1.shape[-1])], -1)
    X_test2 = np.concatenate([imputer.fit_transform(X_test2[..., ii])[:, :, None] for ii in range(X_test2.shape[-1])], -1)
    X_test3 = np.concatenate([imputer.fit_transform(X_test3[..., ii])[:, :, None] for ii in range(X_test3.shape[-1])], -1)
    model = model_name(input_shape=(100, 72), n_classes=1, output_activation=None, verbose=True)
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  metrics=['mean_squared_error', 
                           tf.keras.metrics.RootMeanSquaredError(name='rmse'), 
                           'mean_absolute_error'])
    # ModelCheckpoint
    ckpt = tf.keras.callbacks.ModelCheckpoint(
    ckpt_filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
    )
    # CSVLogger
    log = tf.keras.callbacks.CSVLogger(filename=(str(model_name) + '.csv'), append=True)
    callbacks = [early_stopping, ckpt, reduce_lr, log]
    history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=128, validation_split=0.2,
                        callbacks=callbacks)
    model.load_weights(ckpt_filepath)
    y_predicted1 = model.predict(X_test1)
    y_predicted1 = y_predicted1.reshape(y_test1.shape)
    y_predicted2 = model.predict(X_test2)
    y_predicted2 = y_predicted2.reshape(y_test2.shape)
    y_predicted3 = model.predict(X_test3)
    y_predicted3 = y_predicted3.reshape(y_test3.shape)
    coef1 = np.corrcoef(y_test1, y_predicted1)[0,1] ** 2
    coef2 = np.corrcoef(y_test2, y_predicted2)[0,1] ** 2
    coef3 = np.corrcoef(y_test3, y_predicted3)[0,1] ** 2
    eval_1 = model.evaluate(X_test1, y_test1)
    eval_2 = model.evaluate(X_test2, y_test2)
    eval_3 = model.evaluate(X_test3, y_test3)
    result = {'Model': [name_str]*12,
              'Train_site': [site_1]*12,
              'Test_site': [site_1] * 4 + [site_2] * 4 + [site_3] * 4,
              'Metric': ['MSE', 'RMSE', 'MAE', 'coef'] * 3,
              'Value': [eval_1[1], eval_1[2], eval_1[3], coef1,
                        eval_2[1], eval_2[2], eval_2[3], coef2,
                        eval_3[1], eval_3[2], eval_3[3], coef3]}
    df = pd.DataFrame(result)
    return df, history

### lenet

In [31]:
df_lenet1, history_lenet1 = single_cross_site(cnn_lenet, 'cnn_lenet', '/tmp/lenet.m5', 0.001, 0, 3, 4, X, y)

pooling layers: 4
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d_16 (Conv1D)          (None, 100, 6)            1302      
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 50, 6)            0         
 g1D)                                                            
                                                                 
 conv1d_17 (Conv1D)          (None, 50, 16)            304       
                                                                 
 max_pooling1d_17 (MaxPoolin  (None, 25, 16)           0         
 g1D)                                                            
                                                                 
 conv1d_18 (Conv1D)          (None, 25, 2

Epoch 13/1000
3/4 [=====================>........] - ETA: 0s - loss: 19.9975 - mean_squared_error: 19.9975 - rmse: 4.4719 - mean_absolute_error: 3.5199
Epoch 00013: val_loss did not improve from 10.32761
4/4 [==============================] - 0s 38ms/step - loss: 19.2346 - mean_squared_error: 19.2346 - rmse: 4.3857 - mean_absolute_error: 3.4381 - val_loss: 13.4040 - val_mean_squared_error: 13.4040 - val_rmse: 3.6611 - val_mean_absolute_error: 2.7109 - lr: 0.0010
Epoch 14/1000
4/4 [==============================] - ETA: 0s - loss: 19.1802 - mean_squared_error: 19.1802 - rmse: 4.3795 - mean_absolute_error: 3.4263
Epoch 00014: val_loss did not improve from 10.32761
4/4 [==============================] - 0s 25ms/step - loss: 19.1802 - mean_squared_error: 19.1802 - rmse: 4.3795 - mean_absolute_error: 3.4263 - val_loss: 12.9031 - val_mean_squared_error: 12.9031 - val_rmse: 3.5921 - val_mean_absolute_error: 2.6604 - lr: 0.0010
Epoch 15/1000
1/4 [======>.......................] - ETA: 0s - los

Epoch 31/1000
1/4 [======>.......................] - ETA: 0s - loss: 15.8748 - mean_squared_error: 15.8748 - rmse: 3.9843 - mean_absolute_error: 3.1994
Epoch 00031: val_loss did not improve from 10.32761
4/4 [==============================] - 0s 23ms/step - loss: 15.6499 - mean_squared_error: 15.6499 - rmse: 3.9560 - mean_absolute_error: 3.1092 - val_loss: 11.4605 - val_mean_squared_error: 11.4605 - val_rmse: 3.3853 - val_mean_absolute_error: 2.5374 - lr: 5.0000e-04
Epoch 32/1000
1/4 [======>.......................] - ETA: 0s - loss: 19.8109 - mean_squared_error: 19.8109 - rmse: 4.4509 - mean_absolute_error: 3.3150
Epoch 00032: val_loss did not improve from 10.32761
4/4 [==============================] - 0s 22ms/step - loss: 17.7837 - mean_squared_error: 17.7837 - rmse: 4.2171 - mean_absolute_error: 3.2769 - val_loss: 10.7237 - val_mean_squared_error: 10.7237 - val_rmse: 3.2747 - val_mean_absolute_error: 2.5080 - lr: 5.0000e-04
Epoch 33/1000
1/4 [======>.......................] - ETA: 

4/4 [==============================] - ETA: 0s - loss: 16.2040 - mean_squared_error: 16.2040 - rmse: 4.0254 - mean_absolute_error: 3.2024
Epoch 00048: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 28ms/step - loss: 16.2040 - mean_squared_error: 16.2040 - rmse: 4.0254 - mean_absolute_error: 3.2024 - val_loss: 11.3588 - val_mean_squared_error: 11.3588 - val_rmse: 3.3703 - val_mean_absolute_error: 2.5296 - lr: 5.0000e-04
Epoch 49/1000
1/4 [======>.......................] - ETA: 0s - loss: 15.3402 - mean_squared_error: 15.3402 - rmse: 3.9167 - mean_absolute_error: 3.0691
Epoch 00049: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 23ms/step - loss: 16.1891 - mean_squared_error: 16.1891 - rmse: 4.0236 - mean_absolute_error: 3.2110 - val_loss: 11.1132 - val_mean_squared_error: 11.1132 - val_rmse: 3.3337 - val_mean_absolute_error: 2.5182 - lr: 5.0000e-04
Epoch 50/1000
1/4 [======>.......................] - ETA: 0s - loss: 15.

Epoch 66/1000
4/4 [==============================] - ETA: 0s - loss: 17.0958 - mean_squared_error: 17.0958 - rmse: 4.1347 - mean_absolute_error: 3.2673
Epoch 00066: val_loss did not improve from 10.16643

Epoch 00066: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
4/4 [==============================] - 0s 26ms/step - loss: 17.0958 - mean_squared_error: 17.0958 - rmse: 4.1347 - mean_absolute_error: 3.2673 - val_loss: 11.1002 - val_mean_squared_error: 11.1002 - val_rmse: 3.3317 - val_mean_absolute_error: 2.5136 - lr: 5.0000e-04
Epoch 67/1000
1/4 [======>.......................] - ETA: 0s - loss: 16.3999 - mean_squared_error: 16.3999 - rmse: 4.0497 - mean_absolute_error: 3.0380
Epoch 00067: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 24ms/step - loss: 14.3302 - mean_squared_error: 14.3302 - rmse: 3.7855 - mean_absolute_error: 2.9665 - val_loss: 10.5447 - val_mean_squared_error: 10.5447 - val_rmse: 3.2473 - val_mean_absolute_error: 2

1/4 [======>.......................] - ETA: 0s - loss: 13.3855 - mean_squared_error: 13.3855 - rmse: 3.6586 - mean_absolute_error: 2.8160
Epoch 00083: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 24ms/step - loss: 14.9379 - mean_squared_error: 14.9379 - rmse: 3.8650 - mean_absolute_error: 2.9636 - val_loss: 11.0193 - val_mean_squared_error: 11.0193 - val_rmse: 3.3195 - val_mean_absolute_error: 2.5066 - lr: 2.5000e-04
Epoch 84/1000
1/4 [======>.......................] - ETA: 0s - loss: 14.8828 - mean_squared_error: 14.8828 - rmse: 3.8578 - mean_absolute_error: 3.0050
Epoch 00084: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 23ms/step - loss: 14.2847 - mean_squared_error: 14.2847 - rmse: 3.7795 - mean_absolute_error: 2.9417 - val_loss: 10.5267 - val_mean_squared_error: 10.5267 - val_rmse: 3.2445 - val_mean_absolute_error: 2.4894 - lr: 2.5000e-04
Epoch 85/1000
1/4 [======>.......................] - ETA: 0s - loss: 17.

1/4 [======>.......................] - ETA: 0s - loss: 12.6963 - mean_squared_error: 12.6963 - rmse: 3.5632 - mean_absolute_error: 2.7185
Epoch 00100: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 22ms/step - loss: 14.8188 - mean_squared_error: 14.8188 - rmse: 3.8495 - mean_absolute_error: 3.0407 - val_loss: 10.6364 - val_mean_squared_error: 10.6364 - val_rmse: 3.2613 - val_mean_absolute_error: 2.4909 - lr: 1.2500e-04
Epoch 101/1000
4/4 [==============================] - ETA: 0s - loss: 15.8131 - mean_squared_error: 15.8131 - rmse: 3.9766 - mean_absolute_error: 3.1184
Epoch 00101: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 25ms/step - loss: 15.8131 - mean_squared_error: 15.8131 - rmse: 3.9766 - mean_absolute_error: 3.1184 - val_loss: 10.4976 - val_mean_squared_error: 10.4976 - val_rmse: 3.2400 - val_mean_absolute_error: 2.4861 - lr: 1.2500e-04
Epoch 102/1000
4/4 [==============================] - ETA: 0s - loss: 1

4/4 [==============================] - ETA: 0s - loss: 14.5130 - mean_squared_error: 14.5130 - rmse: 3.8096 - mean_absolute_error: 3.0688
Epoch 00117: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 25ms/step - loss: 14.5130 - mean_squared_error: 14.5130 - rmse: 3.8096 - mean_absolute_error: 3.0688 - val_loss: 10.6056 - val_mean_squared_error: 10.6056 - val_rmse: 3.2566 - val_mean_absolute_error: 2.4870 - lr: 6.2500e-05
Epoch 118/1000
1/4 [======>.......................] - ETA: 0s - loss: 16.5867 - mean_squared_error: 16.5867 - rmse: 4.0727 - mean_absolute_error: 3.2977
Epoch 00118: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 24ms/step - loss: 15.3770 - mean_squared_error: 15.3770 - rmse: 3.9214 - mean_absolute_error: 3.1074 - val_loss: 10.6008 - val_mean_squared_error: 10.6008 - val_rmse: 3.2559 - val_mean_absolute_error: 2.4868 - lr: 6.2500e-05
Epoch 119/1000
1/4 [======>.......................] - ETA: 0s - loss: 1

1/4 [======>.......................] - ETA: 0s - loss: 14.7398 - mean_squared_error: 14.7398 - rmse: 3.8392 - mean_absolute_error: 2.9759
Epoch 00134: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 23ms/step - loss: 15.2385 - mean_squared_error: 15.2385 - rmse: 3.9037 - mean_absolute_error: 3.0815 - val_loss: 10.6607 - val_mean_squared_error: 10.6607 - val_rmse: 3.2651 - val_mean_absolute_error: 2.4874 - lr: 3.1250e-05
Epoch 135/1000
1/4 [======>.......................] - ETA: 0s - loss: 17.6562 - mean_squared_error: 17.6562 - rmse: 4.2019 - mean_absolute_error: 3.1953
Epoch 00135: val_loss did not improve from 10.16643
4/4 [==============================] - 0s 22ms/step - loss: 14.5761 - mean_squared_error: 14.5761 - rmse: 3.8179 - mean_absolute_error: 2.9782 - val_loss: 10.6400 - val_mean_squared_error: 10.6400 - val_rmse: 3.2619 - val_mean_absolute_error: 2.4867 - lr: 3.1250e-05
Epoch 136/1000
1/4 [======>.......................] - ETA: 0s - loss: 1

In [33]:
df_lenet2, history_lenet2 = single_cross_site(cnn_lenet, 'cnn_lenet', '/tmp/lenet.m5', 0.001, 3, 0, 4, X, y)

pooling layers: 4
Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d_20 (Conv1D)          (None, 100, 6)            1302      
                                                                 
 max_pooling1d_20 (MaxPoolin  (None, 50, 6)            0         
 g1D)                                                            
                                                                 
 conv1d_21 (Conv1D)          (None, 50, 16)            304       
                                                                 
 max_pooling1d_21 (MaxPoolin  (None, 25, 16)           0         
 g1D)                                                            
                                                                 
 conv1d_22 (Conv1D)          (None, 25, 2

Epoch 13/1000
1/4 [======>.......................] - ETA: 0s - loss: 19.1181 - mean_squared_error: 19.1181 - rmse: 4.3724 - mean_absolute_error: 3.3425
Epoch 00013: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 23ms/step - loss: 18.1769 - mean_squared_error: 18.1769 - rmse: 4.2634 - mean_absolute_error: 3.3002 - val_loss: 13.2816 - val_mean_squared_error: 13.2816 - val_rmse: 3.6444 - val_mean_absolute_error: 2.6690 - lr: 0.0010
Epoch 14/1000
1/4 [======>.......................] - ETA: 0s - loss: 21.5543 - mean_squared_error: 21.5543 - rmse: 4.6427 - mean_absolute_error: 3.7119
Epoch 00014: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 24ms/step - loss: 17.5411 - mean_squared_error: 17.5411 - rmse: 4.1882 - mean_absolute_error: 3.2593 - val_loss: 13.7641 - val_mean_squared_error: 13.7641 - val_rmse: 3.7100 - val_mean_absolute_error: 2.6981 - lr: 0.0010
Epoch 15/1000
1/4 [======>.......................] - ETA: 0s - los

Epoch 31/1000
1/4 [======>.......................] - ETA: 0s - loss: 14.0081 - mean_squared_error: 14.0081 - rmse: 3.7427 - mean_absolute_error: 3.0074
Epoch 00031: val_loss did not improve from 11.71418

Epoch 00031: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 21ms/step - loss: 16.2056 - mean_squared_error: 16.2056 - rmse: 4.0256 - mean_absolute_error: 3.2271 - val_loss: 12.0130 - val_mean_squared_error: 12.0130 - val_rmse: 3.4660 - val_mean_absolute_error: 2.6276 - lr: 0.0010
Epoch 32/1000
1/4 [======>.......................] - ETA: 0s - loss: 14.3218 - mean_squared_error: 14.3218 - rmse: 3.7844 - mean_absolute_error: 2.9411
Epoch 00032: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 20ms/step - loss: 16.5577 - mean_squared_error: 16.5577 - rmse: 4.0691 - mean_absolute_error: 3.1655 - val_loss: 12.3184 - val_mean_squared_error: 12.3184 - val_rmse: 3.5098 - val_mean_absolute_error: 2.626

1/4 [======>.......................] - ETA: 0s - loss: 16.4013 - mean_squared_error: 16.4013 - rmse: 4.0499 - mean_absolute_error: 3.0686
Epoch 00048: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 21ms/step - loss: 14.0233 - mean_squared_error: 14.0233 - rmse: 3.7448 - mean_absolute_error: 2.9217 - val_loss: 12.3691 - val_mean_squared_error: 12.3691 - val_rmse: 3.5170 - val_mean_absolute_error: 2.6244 - lr: 5.0000e-04
Epoch 49/1000
1/4 [======>.......................] - ETA: 0s - loss: 15.8861 - mean_squared_error: 15.8861 - rmse: 3.9857 - mean_absolute_error: 3.0447
Epoch 00049: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 21ms/step - loss: 16.2467 - mean_squared_error: 16.2467 - rmse: 4.0307 - mean_absolute_error: 3.1404 - val_loss: 12.2834 - val_mean_squared_error: 12.2834 - val_rmse: 3.5048 - val_mean_absolute_error: 2.6169 - lr: 5.0000e-04
Epoch 50/1000
1/4 [======>.......................] - ETA: 0s - loss: 15.

1/4 [======>.......................] - ETA: 0s - loss: 12.6064 - mean_squared_error: 12.6064 - rmse: 3.5505 - mean_absolute_error: 2.7665
Epoch 00065: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 23ms/step - loss: 14.7416 - mean_squared_error: 14.7416 - rmse: 3.8395 - mean_absolute_error: 2.9773 - val_loss: 12.5551 - val_mean_squared_error: 12.5551 - val_rmse: 3.5433 - val_mean_absolute_error: 2.6212 - lr: 2.5000e-04
Epoch 66/1000
1/4 [======>.......................] - ETA: 0s - loss: 14.2518 - mean_squared_error: 14.2518 - rmse: 3.7752 - mean_absolute_error: 2.9055
Epoch 00066: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 21ms/step - loss: 16.1382 - mean_squared_error: 16.1382 - rmse: 4.0172 - mean_absolute_error: 3.0608 - val_loss: 12.2891 - val_mean_squared_error: 12.2891 - val_rmse: 3.5056 - val_mean_absolute_error: 2.6171 - lr: 2.5000e-04
Epoch 67/1000
1/4 [======>.......................] - ETA: 0s - loss: 13.

1/4 [======>.......................] - ETA: 0s - loss: 12.4869 - mean_squared_error: 12.4869 - rmse: 3.5337 - mean_absolute_error: 2.8318
Epoch 00082: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 23ms/step - loss: 13.6250 - mean_squared_error: 13.6250 - rmse: 3.6912 - mean_absolute_error: 2.9073 - val_loss: 11.8878 - val_mean_squared_error: 11.8878 - val_rmse: 3.4479 - val_mean_absolute_error: 2.6316 - lr: 1.2500e-04
Epoch 83/1000
1/4 [======>.......................] - ETA: 0s - loss: 15.4968 - mean_squared_error: 15.4968 - rmse: 3.9366 - mean_absolute_error: 2.9420
Epoch 00083: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 21ms/step - loss: 14.4205 - mean_squared_error: 14.4205 - rmse: 3.7974 - mean_absolute_error: 2.9454 - val_loss: 11.8128 - val_mean_squared_error: 11.8128 - val_rmse: 3.4370 - val_mean_absolute_error: 2.6366 - lr: 1.2500e-04
Epoch 84/1000
1/4 [======>.......................] - ETA: 0s - loss: 17.

1/4 [======>.......................] - ETA: 0s - loss: 10.5593 - mean_squared_error: 10.5593 - rmse: 3.2495 - mean_absolute_error: 2.5910
Epoch 00099: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 21ms/step - loss: 14.4834 - mean_squared_error: 14.4834 - rmse: 3.8057 - mean_absolute_error: 2.9227 - val_loss: 12.0558 - val_mean_squared_error: 12.0558 - val_rmse: 3.4721 - val_mean_absolute_error: 2.6072 - lr: 6.2500e-05
Epoch 100/1000
1/4 [======>.......................] - ETA: 0s - loss: 13.8960 - mean_squared_error: 13.8960 - rmse: 3.7277 - mean_absolute_error: 2.8233
Epoch 00100: val_loss did not improve from 11.71418
4/4 [==============================] - 0s 21ms/step - loss: 13.6119 - mean_squared_error: 13.6119 - rmse: 3.6894 - mean_absolute_error: 2.9083 - val_loss: 12.0085 - val_mean_squared_error: 12.0085 - val_rmse: 3.4653 - val_mean_absolute_error: 2.6082 - lr: 6.2500e-05
Epoch 101/1000
1/4 [======>.......................] - ETA: 0s - loss: 1

In [35]:
df_lenet3, history_lenet3 = single_cross_site(cnn_lenet, 'cnn_lenet', '/tmp/lenet.m5', 0.001, 4, 0, 3, X, y)

pooling layers: 4
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d_24 (Conv1D)          (None, 100, 6)            1302      
                                                                 
 max_pooling1d_24 (MaxPoolin  (None, 50, 6)            0         
 g1D)                                                            
                                                                 
 conv1d_25 (Conv1D)          (None, 50, 16)            304       
                                                                 
 max_pooling1d_25 (MaxPoolin  (None, 25, 16)           0         
 g1D)                                                            
                                                                 
 conv1d_26 (Conv1D)          (None, 25, 2

1/2 [==============>...............] - ETA: 0s - loss: 31.7388 - mean_squared_error: 31.7388 - rmse: 5.6337 - mean_absolute_error: 4.3412
Epoch 00012: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 31ms/step - loss: 32.0512 - mean_squared_error: 32.0512 - rmse: 5.6614 - mean_absolute_error: 4.4322 - val_loss: 31.0547 - val_mean_squared_error: 31.0547 - val_rmse: 5.5727 - val_mean_absolute_error: 4.4722 - lr: 0.0010
Epoch 13/1000
1/2 [==============>...............] - ETA: 0s - loss: 28.2674 - mean_squared_error: 28.2674 - rmse: 5.3167 - mean_absolute_error: 4.2358
Epoch 00013: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 30ms/step - loss: 27.7825 - mean_squared_error: 27.7825 - rmse: 5.2709 - mean_absolute_error: 4.1993 - val_loss: 22.7370 - val_mean_squared_error: 22.7370 - val_rmse: 4.7683 - val_mean_absolute_error: 3.8863 - lr: 0.0010
Epoch 14/1000
1/2 [==============>...............] - ETA: 0s - loss: 28.1706 - m

Epoch 30/1000
1/2 [==============>...............] - ETA: 0s - loss: 25.0847 - mean_squared_error: 25.0847 - rmse: 5.0085 - mean_absolute_error: 3.8657
Epoch 00030: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 32ms/step - loss: 25.1820 - mean_squared_error: 25.1820 - rmse: 5.0182 - mean_absolute_error: 3.9221 - val_loss: 21.4500 - val_mean_squared_error: 21.4500 - val_rmse: 4.6314 - val_mean_absolute_error: 3.7650 - lr: 5.0000e-04
Epoch 31/1000
1/2 [==============>...............] - ETA: 0s - loss: 22.8326 - mean_squared_error: 22.8326 - rmse: 4.7784 - mean_absolute_error: 3.9537
Epoch 00031: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 31ms/step - loss: 22.3744 - mean_squared_error: 22.3744 - rmse: 4.7302 - mean_absolute_error: 3.9068 - val_loss: 21.6073 - val_mean_squared_error: 21.6073 - val_rmse: 4.6484 - val_mean_absolute_error: 3.7765 - lr: 5.0000e-04
Epoch 32/1000
1/2 [==============>...............] - ETA: 

Epoch 48/1000
1/2 [==============>...............] - ETA: 0s - loss: 16.1689 - mean_squared_error: 16.1689 - rmse: 4.0211 - mean_absolute_error: 3.2214
Epoch 00048: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 32ms/step - loss: 15.9391 - mean_squared_error: 15.9391 - rmse: 3.9924 - mean_absolute_error: 3.1827 - val_loss: 20.0891 - val_mean_squared_error: 20.0891 - val_rmse: 4.4821 - val_mean_absolute_error: 3.6330 - lr: 2.5000e-04
Epoch 49/1000
1/2 [==============>...............] - ETA: 0s - loss: 15.8018 - mean_squared_error: 15.8018 - rmse: 3.9751 - mean_absolute_error: 3.2155
Epoch 00049: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 30ms/step - loss: 15.7096 - mean_squared_error: 15.7096 - rmse: 3.9635 - mean_absolute_error: 3.2210 - val_loss: 19.4771 - val_mean_squared_error: 19.4771 - val_rmse: 4.4133 - val_mean_absolute_error: 3.5863 - lr: 2.5000e-04
Epoch 50/1000
1/2 [==============>...............] - ETA: 

2/2 [==============================] - 0s 29ms/step - loss: 18.4533 - mean_squared_error: 18.4533 - rmse: 4.2957 - mean_absolute_error: 3.2805 - val_loss: 18.3528 - val_mean_squared_error: 18.3528 - val_rmse: 4.2840 - val_mean_absolute_error: 3.4734 - lr: 2.5000e-04
Epoch 66/1000
1/2 [==============>...............] - ETA: 0s - loss: 16.4594 - mean_squared_error: 16.4594 - rmse: 4.0570 - mean_absolute_error: 3.1964
Epoch 00066: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 29ms/step - loss: 16.1955 - mean_squared_error: 16.1955 - rmse: 4.0244 - mean_absolute_error: 3.1848 - val_loss: 18.5533 - val_mean_squared_error: 18.5533 - val_rmse: 4.3074 - val_mean_absolute_error: 3.4861 - lr: 2.5000e-04
Epoch 67/1000
1/2 [==============>...............] - ETA: 0s - loss: 15.9205 - mean_squared_error: 15.9205 - rmse: 3.9901 - mean_absolute_error: 3.0911
Epoch 00067: val_loss did not improve from 12.31365

Epoch 00067: ReduceLROnPlateau reducing learning rate to 

Epoch 83/1000
1/2 [==============>...............] - ETA: 0s - loss: 17.3410 - mean_squared_error: 17.3410 - rmse: 4.1643 - mean_absolute_error: 3.2600
Epoch 00083: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 30ms/step - loss: 20.2177 - mean_squared_error: 20.2177 - rmse: 4.4964 - mean_absolute_error: 3.5360 - val_loss: 18.3929 - val_mean_squared_error: 18.3929 - val_rmse: 4.2887 - val_mean_absolute_error: 3.4394 - lr: 1.2500e-04
Epoch 84/1000
1/2 [==============>...............] - ETA: 0s - loss: 17.8778 - mean_squared_error: 17.8778 - rmse: 4.2282 - mean_absolute_error: 3.4828
Epoch 00084: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 29ms/step - loss: 17.2060 - mean_squared_error: 17.2060 - rmse: 4.1480 - mean_absolute_error: 3.4649 - val_loss: 18.0208 - val_mean_squared_error: 18.0208 - val_rmse: 4.2451 - val_mean_absolute_error: 3.4067 - lr: 1.2500e-04
Epoch 85/1000
1/2 [==============>...............] - ETA: 

1/2 [==============>...............] - ETA: 0s - loss: 19.1628 - mean_squared_error: 19.1628 - rmse: 4.3775 - mean_absolute_error: 3.3959
Epoch 00100: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 29ms/step - loss: 19.5406 - mean_squared_error: 19.5406 - rmse: 4.4205 - mean_absolute_error: 3.4260 - val_loss: 18.6803 - val_mean_squared_error: 18.6803 - val_rmse: 4.3221 - val_mean_absolute_error: 3.4540 - lr: 6.2500e-05
Epoch 101/1000
1/2 [==============>...............] - ETA: 0s - loss: 18.4199 - mean_squared_error: 18.4199 - rmse: 4.2918 - mean_absolute_error: 3.4061
Epoch 00101: val_loss did not improve from 12.31365
2/2 [==============================] - 0s 27ms/step - loss: 17.1577 - mean_squared_error: 17.1577 - rmse: 4.1422 - mean_absolute_error: 3.2653 - val_loss: 17.9510 - val_mean_squared_error: 17.9510 - val_rmse: 4.2369 - val_mean_absolute_error: 3.3935 - lr: 6.2500e-05
Epoch 102/1000
1/2 [==============>...............] - ETA: 0s - loss: 1

In [45]:
df_lenet = (df_lenet1.merge(df_lenet2, how='outer')).merge(df_lenet3, how='outer')

### mlp4

In [47]:
df_mlp41, history_mlp41 = single_cross_site(mlp4, 'mlp4', '/tmp/mlp4.m5', 0.001, 0, 3, 4, X, y)
df_mlp42, history_mlp42 = single_cross_site(mlp4, 'mlp4', '/tmp/mlp4.m5', 0.001, 3, 0, 4, X, y)
df_mlp43, history_mlp43 = single_cross_site(mlp4, 'mlp4', '/tmp/mlp4.m5', 0.001, 4, 0, 3, X, y)

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 flatten_7 (Flatten)         (None, 7200)              0         
                                                                 
 dropout_14 (Dropout)        (None, 7200)              0         
                                                                 
 dense_21 (Dense)            (None, 500)               3600500   
                                                                 
 dropout_15 (Dropout)        (None, 500)               0         
                                                                 
 dense_22 (Dense)            (None, 500)               250500    
                                                                 
 dropout_16 (Dropout)        (None, 500)               0   

Epoch 15/1000
4/4 [==============================] - ETA: 0s - loss: 11.4311 - mean_squared_error: 11.4311 - rmse: 3.3810 - mean_absolute_error: 2.6957
Epoch 00015: val_loss did not improve from 10.22209
4/4 [==============================] - 0s 29ms/step - loss: 11.4311 - mean_squared_error: 11.4311 - rmse: 3.3810 - mean_absolute_error: 2.6957 - val_loss: 14.0135 - val_mean_squared_error: 14.0135 - val_rmse: 3.7435 - val_mean_absolute_error: 2.8067 - lr: 0.0010
Epoch 16/1000
4/4 [==============================] - ETA: 0s - loss: 11.2441 - mean_squared_error: 11.2441 - rmse: 3.3532 - mean_absolute_error: 2.6707
Epoch 00016: val_loss did not improve from 10.22209
4/4 [==============================] - 0s 27ms/step - loss: 11.2441 - mean_squared_error: 11.2441 - rmse: 3.3532 - mean_absolute_error: 2.6707 - val_loss: 14.4605 - val_mean_squared_error: 14.4605 - val_rmse: 3.8027 - val_mean_absolute_error: 2.8489 - lr: 0.0010
Epoch 17/1000
4/4 [==============================] - ETA: 0s - los

4/4 [==============================] - ETA: 0s - loss: 10.2015 - mean_squared_error: 10.2015 - rmse: 3.1940 - mean_absolute_error: 2.4979
Epoch 00032: val_loss did not improve from 10.22209
4/4 [==============================] - 0s 25ms/step - loss: 10.2015 - mean_squared_error: 10.2015 - rmse: 3.1940 - mean_absolute_error: 2.4979 - val_loss: 13.8740 - val_mean_squared_error: 13.8740 - val_rmse: 3.7248 - val_mean_absolute_error: 2.7896 - lr: 5.0000e-04
Epoch 33/1000
4/4 [==============================] - ETA: 0s - loss: 10.4513 - mean_squared_error: 10.4513 - rmse: 3.2329 - mean_absolute_error: 2.5817
Epoch 00033: val_loss did not improve from 10.22209
4/4 [==============================] - 0s 27ms/step - loss: 10.4513 - mean_squared_error: 10.4513 - rmse: 3.2329 - mean_absolute_error: 2.5817 - val_loss: 16.4304 - val_mean_squared_error: 16.4304 - val_rmse: 4.0534 - val_mean_absolute_error: 3.0508 - lr: 5.0000e-04
Epoch 34/1000
4/4 [==============================] - ETA: 0s - loss: 9.6

Epoch 50/1000
4/4 [==============================] - ETA: 0s - loss: 9.1496 - mean_squared_error: 9.1496 - rmse: 3.0248 - mean_absolute_error: 2.4552
Epoch 00050: val_loss did not improve from 10.22209
4/4 [==============================] - 0s 27ms/step - loss: 9.1496 - mean_squared_error: 9.1496 - rmse: 3.0248 - mean_absolute_error: 2.4552 - val_loss: 12.0683 - val_mean_squared_error: 12.0683 - val_rmse: 3.4739 - val_mean_absolute_error: 2.6117 - lr: 2.5000e-04
Epoch 51/1000
4/4 [==============================] - ETA: 0s - loss: 8.3332 - mean_squared_error: 8.3332 - rmse: 2.8867 - mean_absolute_error: 2.2952
Epoch 00051: val_loss did not improve from 10.22209
4/4 [==============================] - 0s 25ms/step - loss: 8.3332 - mean_squared_error: 8.3332 - rmse: 2.8867 - mean_absolute_error: 2.2952 - val_loss: 14.0089 - val_mean_squared_error: 14.0089 - val_rmse: 3.7428 - val_mean_absolute_error: 2.8023 - lr: 2.5000e-04
Epoch 52/1000
4/4 [==============================] - ETA: 0s - los

4/4 [==============================] - 0s 90ms/step - loss: 7.6381 - mean_squared_error: 7.6381 - rmse: 2.7637 - mean_absolute_error: 2.1867 - val_loss: 10.1314 - val_mean_squared_error: 10.1314 - val_rmse: 3.1830 - val_mean_absolute_error: 2.4142 - lr: 1.2500e-04
Epoch 68/1000
4/4 [==============================] - ETA: 0s - loss: 7.7278 - mean_squared_error: 7.7278 - rmse: 2.7799 - mean_absolute_error: 2.2512
Epoch 00068: val_loss did not improve from 10.13142
4/4 [==============================] - 0s 28ms/step - loss: 7.7278 - mean_squared_error: 7.7278 - rmse: 2.7799 - mean_absolute_error: 2.2512 - val_loss: 12.4846 - val_mean_squared_error: 12.4846 - val_rmse: 3.5334 - val_mean_absolute_error: 2.6355 - lr: 1.2500e-04
Epoch 69/1000
4/4 [==============================] - ETA: 0s - loss: 7.6099 - mean_squared_error: 7.6099 - rmse: 2.7586 - mean_absolute_error: 2.1709
Epoch 00069: val_loss did not improve from 10.13142
4/4 [==============================] - 0s 29ms/step - loss: 7.6099

...

1/2 [==============>...............] - ETA: 0s - loss: 11.3037 - mean_squared_error: 11.3037 - rmse: 3.3621 - mean_absolute_error: 2.6591
Epoch 00054: val_loss did not improve from 9.78453
2/2 [==============================] - 0s 37ms/step - loss: 11.8104 - mean_squared_error: 11.8104 - rmse: 3.4366 - mean_absolute_error: 2.7419 - val_loss: 14.3906 - val_mean_squared_error: 14.3906 - val_rmse: 3.7935 - val_mean_absolute_error: 3.0611 - lr: 2.5000e-04
Epoch 55/1000
1/2 [==============>...............] - ETA: 0s - loss: 12.4168 - mean_squared_error: 12.4168 - rmse: 3.5237 - mean_absolute_error: 2.8161
Epoch 00055: val_loss did not improve from 9.78453
2/2 [==============================] - 0s 37ms/step - loss: 12.4288 - mean_squared_error: 12.4288 - rmse: 3.5254 - mean_absolute_error: 2.8613 - val_loss: 14.7177 - val_mean_squared_error: 14.7177 - val_rmse: 3.8364 - val_mean_absolute_error: 3.0998 - lr: 2.5000e-04
Epoch 56/1000
1/2 [==============>...............] - ETA: 0s - loss: 12.33

1/2 [==============>...............] - ETA: 0s - loss: 12.8954 - mean_squared_error: 12.8954 - rmse: 3.5910 - mean_absolute_error: 2.9085
Epoch 00071: val_loss did not improve from 9.78453
2/2 [==============================] - 0s 58ms/step - loss: 12.1375 - mean_squared_error: 12.1375 - rmse: 3.4839 - mean_absolute_error: 2.8074 - val_loss: 12.1410 - val_mean_squared_error: 12.1410 - val_rmse: 3.4844 - val_mean_absolute_error: 2.8139 - lr: 1.2500e-04
Epoch 72/1000
1/2 [==============>...............] - ETA: 0s - loss: 11.7334 - mean_squared_error: 11.7334 - rmse: 3.4254 - mean_absolute_error: 2.9108
Epoch 00072: val_loss did not improve from 9.78453
2/2 [==============================] - 0s 42ms/step - loss: 11.3814 - mean_squared_error: 11.3814 - rmse: 3.3736 - mean_absolute_error: 2.8490 - val_loss: 13.3710 - val_mean_squared_error: 13.3710 - val_rmse: 3.6566 - val_mean_absolute_error: 2.9581 - lr: 1.2500e-04
Epoch 73/1000
1/2 [==============>...............] - ETA: 0s - loss: 12.02

1/2 [==============>...............] - ETA: 0s - loss: 12.5912 - mean_squared_error: 12.5912 - rmse: 3.5484 - mean_absolute_error: 2.8321
Epoch 00088: val_loss did not improve from 9.78453
2/2 [==============================] - 0s 40ms/step - loss: 12.3386 - mean_squared_error: 12.3386 - rmse: 3.5126 - mean_absolute_error: 2.8389 - val_loss: 12.1280 - val_mean_squared_error: 12.1280 - val_rmse: 3.4825 - val_mean_absolute_error: 2.8165 - lr: 6.2500e-05
Epoch 89/1000
1/2 [==============>...............] - ETA: 0s - loss: 13.8302 - mean_squared_error: 13.8302 - rmse: 3.7189 - mean_absolute_error: 3.0306
Epoch 00089: val_loss did not improve from 9.78453
2/2 [==============================] - 0s 37ms/step - loss: 12.3821 - mean_squared_error: 12.3821 - rmse: 3.5188 - mean_absolute_error: 2.8453 - val_loss: 12.8333 - val_mean_squared_error: 12.8333 - val_rmse: 3.5824 - val_mean_absolute_error: 2.8998 - lr: 6.2500e-05
Epoch 90/1000
1/2 [==============>...............] - ETA: 0s - loss: 11.37

1/2 [==============>...............] - ETA: 0s - loss: 12.7925 - mean_squared_error: 12.7925 - rmse: 3.5767 - mean_absolute_error: 2.9302
Epoch 00105: val_loss did not improve from 9.78453
2/2 [==============================] - 0s 37ms/step - loss: 11.7461 - mean_squared_error: 11.7461 - rmse: 3.4273 - mean_absolute_error: 2.7751 - val_loss: 13.5370 - val_mean_squared_error: 13.5370 - val_rmse: 3.6793 - val_mean_absolute_error: 2.9860 - lr: 3.1250e-05
Epoch 106/1000
1/2 [==============>...............] - ETA: 0s - loss: 10.8428 - mean_squared_error: 10.8428 - rmse: 3.2928 - mean_absolute_error: 2.6983
Epoch 00106: val_loss did not improve from 9.78453
2/2 [==============================] - 0s 36ms/step - loss: 11.1802 - mean_squared_error: 11.1802 - rmse: 3.3437 - mean_absolute_error: 2.7356 - val_loss: 13.3154 - val_mean_squared_error: 13.3154 - val_rmse: 3.6490 - val_mean_absolute_error: 2.9604 - lr: 3.1250e-05
Epoch 107/1000
1/2 [==============>...............] - ETA: 0s - loss: 10.

Epoch 123/1000
1/2 [==============>...............] - ETA: 0s - loss: 11.3512 - mean_squared_error: 11.3512 - rmse: 3.3692 - mean_absolute_error: 2.7177
Epoch 00123: val_loss did not improve from 9.78453

Epoch 00123: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
5/5 [==============================] - 0s 3ms/step - loss: 10.4286 - mean_squared_error: 10.4286 - rmse: 3.2293 - mean_absolute_error: 2.4711


In [53]:
df_mlp4 = (df_mlp41.merge(df_mlp42, how='outer')).merge(df_mlp43, how='outer')

### cnn_vgg

In [55]:
df_vgg1, history_vgg1 = single_cross_site(cnn_vgg, 'cnn_vgg', '/tmp/vgg.m5', 0.001, 0, 3, 4, X, y)
df_vgg2, history_vgg2 = single_cross_site(cnn_vgg, 'cnn_vgg', '/tmp/vgg.m5', 0.001, 3, 0, 4, X, y)
df_vgg3, history_vgg3 = single_cross_site(cnn_vgg, 'cnn_vgg', '/tmp/vgg.m5', 0.001, 4, 0, 3, X, y)

pooling layers: 4
Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 100, 72)]         0         
                                                                 
 conv1d_28 (Conv1D)          (None, 100, 64)           13888     
                                                                 
 conv1d_29 (Conv1D)          (None, 100, 64)           12352     
                                                                 
 max_pooling1d_28 (MaxPoolin  (None, 50, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_30 (Conv1D)          (None, 50, 128)           24704     
                                                                 
 conv1d_31 (Conv1D)          (None, 50, 128)           49280     
                                        

Epoch 11/1000
4/4 [==============================] - ETA: 0s - loss: 11.7811 - mean_squared_error: 11.7811 - rmse: 3.4324 - mean_absolute_error: 2.8231
Epoch 00011: val_loss improved from 13.03373 to 12.92871, saving model to /tmp/vgg.m5
4/4 [==============================] - 3s 887ms/step - loss: 11.7811 - mean_squared_error: 11.7811 - rmse: 3.4324 - mean_absolute_error: 2.8231 - val_loss: 12.9287 - val_mean_squared_error: 12.9287 - val_rmse: 3.5957 - val_mean_absolute_error: 2.9299 - lr: 0.0010
Epoch 12/1000
4/4 [==============================] - ETA: 0s - loss: 10.9943 - mean_squared_error: 10.9943 - rmse: 3.3158 - mean_absolute_error: 2.6448
Epoch 00012: val_loss did not improve from 12.92871
4/4 [==============================] - 2s 482ms/step - loss: 10.9943 - mean_squared_error: 10.9943 - rmse: 3.3158 - mean_absolute_error: 2.6448 - val_loss: 13.3017 - val_mean_squared_error: 13.3017 - val_rmse: 3.6471 - val_mean_absolute_error: 2.8127 - lr: 0.0010
Epoch 13/1000
4/4 [===========

Epoch 28/1000
4/4 [==============================] - ETA: 0s - loss: 8.9085 - mean_squared_error: 8.9085 - rmse: 2.9847 - mean_absolute_error: 2.3958
Epoch 00028: val_loss did not improve from 9.85026
4/4 [==============================] - 2s 500ms/step - loss: 8.9085 - mean_squared_error: 8.9085 - rmse: 2.9847 - mean_absolute_error: 2.3958 - val_loss: 10.7217 - val_mean_squared_error: 10.7217 - val_rmse: 3.2744 - val_mean_absolute_error: 2.5190 - lr: 0.0010
Epoch 29/1000
4/4 [==============================] - ETA: 0s - loss: 7.7813 - mean_squared_error: 7.7813 - rmse: 2.7895 - mean_absolute_error: 2.2086
Epoch 00029: val_loss improved from 9.85026 to 8.70546, saving model to /tmp/vgg.m5
4/4 [==============================] - 3s 918ms/step - loss: 7.7813 - mean_squared_error: 7.7813 - rmse: 2.7895 - mean_absolute_error: 2.2086 - val_loss: 8.7055 - val_mean_squared_error: 8.7055 - val_rmse: 2.9505 - val_mean_absolute_error: 2.3655 - lr: 0.0010
Epoch 30/1000
4/4 [========================

Epoch 46/1000
4/4 [==============================] - ETA: 0s - loss: 4.5314 - mean_squared_error: 4.5314 - rmse: 2.1287 - mean_absolute_error: 1.6781
Epoch 00046: val_loss did not improve from 6.98223
4/4 [==============================] - 2s 498ms/step - loss: 4.5314 - mean_squared_error: 4.5314 - rmse: 2.1287 - mean_absolute_error: 1.6781 - val_loss: 7.4607 - val_mean_squared_error: 7.4607 - val_rmse: 2.7314 - val_mean_absolute_error: 2.0916 - lr: 0.0010
Epoch 47/1000
4/4 [==============================] - ETA: 0s - loss: 4.4408 - mean_squared_error: 4.4408 - rmse: 2.1073 - mean_absolute_error: 1.6337
Epoch 00047: val_loss improved from 6.98223 to 6.62848, saving model to /tmp/vgg.m5
4/4 [==============================] - 3s 900ms/step - loss: 4.4408 - mean_squared_error: 4.4408 - rmse: 2.1073 - mean_absolute_error: 1.6337 - val_loss: 6.6285 - val_mean_squared_error: 6.6285 - val_rmse: 2.5746 - val_mean_absolute_error: 1.9974 - lr: 0.0010
Epoch 48/1000
4/4 [==========================

Epoch 64/1000
4/4 [==============================] - ETA: 0s - loss: 2.5980 - mean_squared_error: 2.5980 - rmse: 1.6118 - mean_absolute_error: 1.2772
Epoch 00064: val_loss did not improve from 5.30776
4/4 [==============================] - 2s 528ms/step - loss: 2.5980 - mean_squared_error: 2.5980 - rmse: 1.6118 - mean_absolute_error: 1.2772 - val_loss: 5.7704 - val_mean_squared_error: 5.7704 - val_rmse: 2.4022 - val_mean_absolute_error: 1.8845 - lr: 0.0010
Epoch 65/1000
4/4 [==============================] - ETA: 0s - loss: 2.4258 - mean_squared_error: 2.4258 - rmse: 1.5575 - mean_absolute_error: 1.2152
Epoch 00065: val_loss did not improve from 5.30776
4/4 [==============================] - 2s 528ms/step - loss: 2.4258 - mean_squared_error: 2.4258 - rmse: 1.5575 - mean_absolute_error: 1.2152 - val_loss: 5.3339 - val_mean_squared_error: 5.3339 - val_rmse: 2.3095 - val_mean_absolute_error: 1.8020 - lr: 0.0010
Epoch 66/1000
4/4 [==============================] - ETA: 0s - loss: 2.8378 - 

...

Epoch 233/1000
2/2 [==============================] - ETA: 0s - loss: 0.5905 - mean_squared_error: 0.5905 - rmse: 0.7684 - mean_absolute_error: 0.5852
Epoch 00233: val_loss did not improve from 7.41781
2/2 [==============================] - 1s 272ms/step - loss: 0.5905 - mean_squared_error: 0.5905 - rmse: 0.7684 - mean_absolute_error: 0.5852 - val_loss: 7.7966 - val_mean_squared_error: 7.7966 - val_rmse: 2.7922 - val_mean_absolute_error: 2.1803 - lr: 6.2500e-05
Epoch 234/1000
2/2 [==============================] - ETA: 0s - loss: 0.6804 - mean_squared_error: 0.6804 - rmse: 0.8248 - mean_absolute_error: 0.6345
Epoch 00234: val_loss did not improve from 7.41781
2/2 [==============================] - 1s 263ms/step - loss: 0.6804 - mean_squared_error: 0.6804 - rmse: 0.8248 - mean_absolute_error: 0.6345 - val_loss: 7.9256 - val_mean_squared_error: 7.9256 - val_rmse: 2.8153 - val_mean_absolute_error: 2.1859 - lr: 6.2500e-05
Epoch 235/1000
2/2 [==============================] - ETA: 0s - loss

Epoch 251/1000
2/2 [==============================] - ETA: 0s - loss: 0.5425 - mean_squared_error: 0.5425 - rmse: 0.7365 - mean_absolute_error: 0.5668
Epoch 00251: val_loss did not improve from 7.41781
2/2 [==============================] - 1s 255ms/step - loss: 0.5425 - mean_squared_error: 0.5425 - rmse: 0.7365 - mean_absolute_error: 0.5668 - val_loss: 8.1542 - val_mean_squared_error: 8.1542 - val_rmse: 2.8555 - val_mean_absolute_error: 2.2048 - lr: 3.1250e-05
Epoch 252/1000
2/2 [==============================] - ETA: 0s - loss: 0.4854 - mean_squared_error: 0.4854 - rmse: 0.6967 - mean_absolute_error: 0.5340
Epoch 00252: val_loss did not improve from 7.41781
2/2 [==============================] - 1s 264ms/step - loss: 0.4854 - mean_squared_error: 0.4854 - rmse: 0.6967 - mean_absolute_error: 0.5340 - val_loss: 7.9702 - val_mean_squared_error: 7.9702 - val_rmse: 2.8232 - val_mean_absolute_error: 2.1922 - lr: 3.1250e-05
Epoch 253/1000
2/2 [==============================] - ETA: 0s - loss

Epoch 269/1000
2/2 [==============================] - ETA: 0s - loss: 0.7779 - mean_squared_error: 0.7779 - rmse: 0.8820 - mean_absolute_error: 0.6525
Epoch 00269: val_loss did not improve from 7.41781
2/2 [==============================] - 1s 268ms/step - loss: 0.7779 - mean_squared_error: 0.7779 - rmse: 0.8820 - mean_absolute_error: 0.6525 - val_loss: 7.9141 - val_mean_squared_error: 7.9141 - val_rmse: 2.8132 - val_mean_absolute_error: 2.1900 - lr: 1.5625e-05
Epoch 270/1000
2/2 [==============================] - ETA: 0s - loss: 0.4376 - mean_squared_error: 0.4376 - rmse: 0.6615 - mean_absolute_error: 0.5150
Epoch 00270: val_loss did not improve from 7.41781
2/2 [==============================] - 1s 255ms/step - loss: 0.4376 - mean_squared_error: 0.4376 - rmse: 0.6615 - mean_absolute_error: 0.5150 - val_loss: 7.9228 - val_mean_squared_error: 7.9228 - val_rmse: 2.8148 - val_mean_absolute_error: 2.1903 - lr: 1.5625e-05
Epoch 271/1000
2/2 [==============================] - ETA: 0s - loss

Epoch 287/1000
2/2 [==============================] - ETA: 0s - loss: 0.4871 - mean_squared_error: 0.4871 - rmse: 0.6979 - mean_absolute_error: 0.5395
Epoch 00287: val_loss did not improve from 7.41781
2/2 [==============================] - 1s 256ms/step - loss: 0.4871 - mean_squared_error: 0.4871 - rmse: 0.6979 - mean_absolute_error: 0.5395 - val_loss: 7.8313 - val_mean_squared_error: 7.8313 - val_rmse: 2.7984 - val_mean_absolute_error: 2.1781 - lr: 7.8125e-06
Epoch 288/1000
2/2 [==============================] - ETA: 0s - loss: 0.5673 - mean_squared_error: 0.5673 - rmse: 0.7532 - mean_absolute_error: 0.5759
Epoch 00288: val_loss did not improve from 7.41781
2/2 [==============================] - 1s 254ms/step - loss: 0.5673 - mean_squared_error: 0.5673 - rmse: 0.7532 - mean_absolute_error: 0.5759 - val_loss: 7.8267 - val_mean_squared_error: 7.8267 - val_rmse: 2.7976 - val_mean_absolute_error: 2.1772 - lr: 7.8125e-06
Epoch 289/1000
2/2 [==============================] - ETA: 0s - loss

5/5 [==============================] - 0s 24ms/step - loss: 10.3192 - mean_squared_error: 10.3192 - rmse: 3.2123 - mean_absolute_error: 2.5538


In [56]:
df_vgg = (df_vgg1.merge(df_vgg2, how='outer')).merge(df_vgg3, how='outer')

### cnn_resnet

In [57]:
df_resnet1, history_resnet1 = single_cross_site(cnn_resnet, 'cnn_resnet', '/tmp/resnet.m5', 0.01, 0, 3, 4, X, y)
df_resnet2, history_resnet2 = single_cross_site(cnn_resnet, 'cnn_resnet', '/tmp/resnet.m5', 0.01, 3, 0, 4, X, y)
df_resnet3, history_resnet3 = single_cross_site(cnn_resnet, 'cnn_resnet', '/tmp/resnet.m5', 0.01, 4, 0, 3, X, y)

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 100, 72)]    0           []                               
                                                                                                  
 conv1d_58 (Conv1D)             (None, 100, 64)      36928       ['input_14[0][0]']               
                                                                                                  
 batch_normalization (BatchNorm  (None, 100, 64)     256         ['conv1d_58[0][0]']              
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 100, 64)      0           ['batch_normalization[0][0

4/4 [==============================] - ETA: 0s - loss: 76.0902 - mean_squared_error: 76.0902 - rmse: 8.7230 - mean_absolute_error: 7.8477  
Epoch 00001: val_loss improved from inf to 103727048.00000, saving model to /tmp/resnet.m5
4/4 [==============================] - 4s 435ms/step - loss: 76.0902 - mean_squared_error: 76.0902 - rmse: 8.7230 - mean_absolute_error: 7.8477 - val_loss: 103727048.0000 - val_mean_squared_error: 103727048.0000 - val_rmse: 10184.6475 - val_mean_absolute_error: 10170.4141 - lr: 0.0100
Epoch 2/1000
4/4 [==============================] - ETA: 0s - loss: 13.0250 - mean_squared_error: 13.0250 - rmse: 3.6090 - mean_absolute_error: 2.8289
Epoch 00002: val_loss did not improve from 103727048.00000
4/4 [==============================] - 1s 322ms/step - loss: 13.0250 - mean_squared_error: 13.0250 - rmse: 3.6090 - mean_absolute_error: 2.8289 - val_loss: 2597715712.0000 - val_mean_squared_error: 2597715712.0000 - val_rmse: 50967.7891 - val_mean_absolute_error: 50882.781

Epoch 17/1000
4/4 [==============================] - ETA: 0s - loss: 1.9548 - mean_squared_error: 1.9548 - rmse: 1.3981 - mean_absolute_error: 1.0808
Epoch 00017: val_loss improved from 40547.77344 to 18571.37891, saving model to /tmp/resnet.m5
4/4 [==============================] - 1s 358ms/step - loss: 1.9548 - mean_squared_error: 1.9548 - rmse: 1.3981 - mean_absolute_error: 1.0808 - val_loss: 18571.3789 - val_mean_squared_error: 18571.3789 - val_rmse: 136.2768 - val_mean_absolute_error: 135.1494 - lr: 0.0100
Epoch 18/1000
4/4 [==============================] - ETA: 0s - loss: 1.7039 - mean_squared_error: 1.7039 - rmse: 1.3053 - mean_absolute_error: 1.0024
Epoch 00018: val_loss improved from 18571.37891 to 8476.20996, saving model to /tmp/resnet.m5
4/4 [==============================] - 1s 351ms/step - loss: 1.7039 - mean_squared_error: 1.7039 - rmse: 1.3053 - mean_absolute_error: 1.0024 - val_loss: 8476.2100 - val_mean_squared_error: 8476.2100 - val_rmse: 92.0663 - val_mean_absolute

Epoch 34/1000
4/4 [==============================] - ETA: 0s - loss: 0.8724 - mean_squared_error: 0.8724 - rmse: 0.9340 - mean_absolute_error: 0.6996
Epoch 00034: val_loss did not improve from 16.49914
4/4 [==============================] - 1s 322ms/step - loss: 0.8724 - mean_squared_error: 0.8724 - rmse: 0.9340 - mean_absolute_error: 0.6996 - val_loss: 37.2001 - val_mean_squared_error: 37.2001 - val_rmse: 6.0992 - val_mean_absolute_error: 5.5516 - lr: 0.0100
Epoch 35/1000
4/4 [==============================] - ETA: 0s - loss: 0.7369 - mean_squared_error: 0.7369 - rmse: 0.8584 - mean_absolute_error: 0.6533
Epoch 00035: val_loss did not improve from 16.49914
4/4 [==============================] - 1s 321ms/step - loss: 0.7369 - mean_squared_error: 0.7369 - rmse: 0.8584 - mean_absolute_error: 0.6533 - val_loss: 29.7166 - val_mean_squared_error: 29.7166 - val_rmse: 5.4513 - val_mean_absolute_error: 4.8345 - lr: 0.0100
Epoch 36/1000
4/4 [==============================] - ETA: 0s - loss: 0.8

Epoch 52/1000
4/4 [==============================] - ETA: 0s - loss: 0.4858 - mean_squared_error: 0.4858 - rmse: 0.6970 - mean_absolute_error: 0.5180
Epoch 00052: val_loss improved from 6.87230 to 4.30084, saving model to /tmp/resnet.m5
4/4 [==============================] - 1s 349ms/step - loss: 0.4858 - mean_squared_error: 0.4858 - rmse: 0.6970 - mean_absolute_error: 0.5180 - val_loss: 4.3008 - val_mean_squared_error: 4.3008 - val_rmse: 2.0738 - val_mean_absolute_error: 1.5619 - lr: 0.0100
Epoch 53/1000
4/4 [==============================] - ETA: 0s - loss: 0.4124 - mean_squared_error: 0.4124 - rmse: 0.6422 - mean_absolute_error: 0.4917
Epoch 00053: val_loss did not improve from 4.30084
4/4 [==============================] - 1s 324ms/step - loss: 0.4124 - mean_squared_error: 0.4124 - rmse: 0.6422 - mean_absolute_error: 0.4917 - val_loss: 30.5913 - val_mean_squared_error: 30.5913 - val_rmse: 5.5309 - val_mean_absolute_error: 5.1042 - lr: 0.0100
Epoch 54/1000
4/4 [=====================

...

Epoch 337/1000
2/2 [==============================] - ETA: 0s - loss: 0.0408 - mean_squared_error: 0.0408 - rmse: 0.2020 - mean_absolute_error: 0.1493
Epoch 00337: val_loss did not improve from 4.29693
2/2 [==============================] - 0s 150ms/step - loss: 0.0408 - mean_squared_error: 0.0408 - rmse: 0.2020 - mean_absolute_error: 0.1493 - val_loss: 4.3299 - val_mean_squared_error: 4.3299 - val_rmse: 2.0808 - val_mean_absolute_error: 1.7425 - lr: 4.8828e-06
Epoch 338/1000
2/2 [==============================] - ETA: 0s - loss: 0.5549 - mean_squared_error: 0.5549 - rmse: 0.7449 - mean_absolute_error: 0.5618
Epoch 00338: val_loss did not improve from 4.29693
2/2 [==============================] - 0s 151ms/step - loss: 0.5549 - mean_squared_error: 0.5549 - rmse: 0.7449 - mean_absolute_error: 0.5618 - val_loss: 4.3256 - val_mean_squared_error: 4.3256 - val_rmse: 2.0798 - val_mean_absolute_error: 1.7419 - lr: 4.8828e-06
Epoch 339/1000
2/2 [==============================] - ETA: 0s - loss

Epoch 355/1000
2/2 [==============================] - ETA: 0s - loss: 0.1578 - mean_squared_error: 0.1578 - rmse: 0.3972 - mean_absolute_error: 0.3168
Epoch 00355: val_loss did not improve from 4.29693
2/2 [==============================] - 0s 145ms/step - loss: 0.1578 - mean_squared_error: 0.1578 - rmse: 0.3972 - mean_absolute_error: 0.3168 - val_loss: 4.3556 - val_mean_squared_error: 4.3556 - val_rmse: 2.0870 - val_mean_absolute_error: 1.7512 - lr: 2.4414e-06
Epoch 356/1000
2/2 [==============================] - ETA: 0s - loss: 0.1041 - mean_squared_error: 0.1041 - rmse: 0.3226 - mean_absolute_error: 0.2470
Epoch 00356: val_loss did not improve from 4.29693
2/2 [==============================] - 0s 148ms/step - loss: 0.1041 - mean_squared_error: 0.1041 - rmse: 0.3226 - mean_absolute_error: 0.2470 - val_loss: 4.3539 - val_mean_squared_error: 4.3539 - val_rmse: 2.0866 - val_mean_absolute_error: 1.7509 - lr: 2.4414e-06
Epoch 357/1000
2/2 [==============================] - ETA: 0s - loss

Epoch 373/1000
2/2 [==============================] - ETA: 0s - loss: 0.2819 - mean_squared_error: 0.2819 - rmse: 0.5310 - mean_absolute_error: 0.3597
Epoch 00373: val_loss did not improve from 4.29693
2/2 [==============================] - 0s 149ms/step - loss: 0.2819 - mean_squared_error: 0.2819 - rmse: 0.5310 - mean_absolute_error: 0.3597 - val_loss: 4.3523 - val_mean_squared_error: 4.3523 - val_rmse: 2.0862 - val_mean_absolute_error: 1.7537 - lr: 1.2207e-06
Epoch 374/1000
2/2 [==============================] - ETA: 0s - loss: 0.0345 - mean_squared_error: 0.0345 - rmse: 0.1857 - mean_absolute_error: 0.1467
Epoch 00374: val_loss did not improve from 4.29693
2/2 [==============================] - 0s 149ms/step - loss: 0.0345 - mean_squared_error: 0.0345 - rmse: 0.1857 - mean_absolute_error: 0.1467 - val_loss: 4.3491 - val_mean_squared_error: 4.3491 - val_rmse: 2.0854 - val_mean_absolute_error: 1.7532 - lr: 1.2207e-06
Epoch 375/1000
2/2 [==============================] - ETA: 0s - loss

Epoch 391/1000
2/2 [==============================] - ETA: 0s - loss: 0.0277 - mean_squared_error: 0.0277 - rmse: 0.1666 - mean_absolute_error: 0.1095
Epoch 00391: val_loss did not improve from 4.29693
2/2 [==============================] - 0s 158ms/step - loss: 0.0277 - mean_squared_error: 0.0277 - rmse: 0.1666 - mean_absolute_error: 0.1095 - val_loss: 4.3364 - val_mean_squared_error: 4.3364 - val_rmse: 2.0824 - val_mean_absolute_error: 1.7516 - lr: 6.1035e-07
Epoch 392/1000
2/2 [==============================] - ETA: 0s - loss: 0.0337 - mean_squared_error: 0.0337 - rmse: 0.1837 - mean_absolute_error: 0.1441
Epoch 00392: val_loss did not improve from 4.29693
2/2 [==============================] - 0s 152ms/step - loss: 0.0337 - mean_squared_error: 0.0337 - rmse: 0.1837 - mean_absolute_error: 0.1441 - val_loss: 4.3352 - val_mean_squared_error: 4.3352 - val_rmse: 2.0821 - val_mean_absolute_error: 1.7514 - lr: 6.1035e-07
Epoch 393/1000
2/2 [==============================] - ETA: 0s - loss

5/5 [==============================] - 0s 19ms/step - loss: 15.4373 - mean_squared_error: 15.4373 - rmse: 3.9290 - mean_absolute_error: 3.0601


In [58]:
df_resnet = (df_resnet1.merge(df_resnet2, how='outer')).merge(df_resnet3, how='outer')

### lstm1v0

In [59]:
df_lstm1v01, history_lstm1v01 = single_cross_site(lstm1v0, 'lstm1v0', '/tmp/lstm1v0.m5', 0.01, 0, 3, 4, X, y)
df_lstm1v02, history_lstm1v02 = single_cross_site(lstm1v0, 'lstm1v0', '/tmp/lstm1v0.m5', 0.01, 3, 0, 4, X, y)
df_lstm1v03, history_lstm1v03 = single_cross_site(lstm1v0, 'lstm1v0', '/tmp/lstm1v0.m5', 0.01, 4, 0, 3, X, y)

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 100, 72)]         0         
                                                                 
 lstm (LSTM)                 (None, 100)               69200     
                                                                 
 dense_45 (Dense)            (None, 1)                 101       
                                                                 
Total params: 69,301
Trainable params: 69,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
4/4 [==============================] - ETA: 0s - loss: 49.5851 - mean_squared_error: 49.5851 - rmse: 7.0417 - mean_absolute_error: 5.5694  
Epoch 00001: val_loss improved from inf to 10.69054, saving model to /tmp/lstm1v0.m5
4/4 [==============================] - 2s 235ms/step - loss: 49.5851 - mean_squa

Epoch 17/1000
4/4 [==============================] - ETA: 0s - loss: 13.1281 - mean_squared_error: 13.1281 - rmse: 3.6233 - mean_absolute_error: 2.9810
Epoch 00017: val_loss did not improve from 9.94931
4/4 [==============================] - 0s 125ms/step - loss: 13.1281 - mean_squared_error: 13.1281 - rmse: 3.6233 - mean_absolute_error: 2.9810 - val_loss: 9.9929 - val_mean_squared_error: 9.9929 - val_rmse: 3.1612 - val_mean_absolute_error: 2.4558 - lr: 0.0100
Epoch 18/1000
4/4 [==============================] - ETA: 0s - loss: 13.1538 - mean_squared_error: 13.1538 - rmse: 3.6268 - mean_absolute_error: 3.0095
Epoch 00018: val_loss did not improve from 9.94931
4/4 [==============================] - 1s 130ms/step - loss: 13.1538 - mean_squared_error: 13.1538 - rmse: 3.6268 - mean_absolute_error: 3.0095 - val_loss: 9.9854 - val_mean_squared_error: 9.9854 - val_rmse: 3.1600 - val_mean_absolute_error: 2.4543 - lr: 0.0100
Epoch 19/1000
4/4 [==============================] - ETA: 0s - loss: 1

4/4 [==============================] - ETA: 0s - loss: 12.7023 - mean_squared_error: 12.7023 - rmse: 3.5640 - mean_absolute_error: 2.8677
Epoch 00034: val_loss improved from 9.61004 to 9.60037, saving model to /tmp/lstm1v0.m5
4/4 [==============================] - 0s 127ms/step - loss: 12.7023 - mean_squared_error: 12.7023 - rmse: 3.5640 - mean_absolute_error: 2.8677 - val_loss: 9.6004 - val_mean_squared_error: 9.6004 - val_rmse: 3.0984 - val_mean_absolute_error: 2.3862 - lr: 0.0100
Epoch 35/1000
4/4 [==============================] - ETA: 0s - loss: 12.8333 - mean_squared_error: 12.8333 - rmse: 3.5824 - mean_absolute_error: 2.9980
Epoch 00035: val_loss improved from 9.60037 to 9.36039, saving model to /tmp/lstm1v0.m5
4/4 [==============================] - 1s 135ms/step - loss: 12.8333 - mean_squared_error: 12.8333 - rmse: 3.5824 - mean_absolute_error: 2.9980 - val_loss: 9.3604 - val_mean_squared_error: 9.3604 - val_rmse: 3.0595 - val_mean_absolute_error: 2.3973 - lr: 0.0100
Epoch 36/1

Epoch 51/1000
4/4 [==============================] - ETA: 0s - loss: 9.2655 - mean_squared_error: 9.2655 - rmse: 3.0439 - mean_absolute_error: 2.4713
Epoch 00051: val_loss improved from 6.81758 to 6.03144, saving model to /tmp/lstm1v0.m5
4/4 [==============================] - 0s 118ms/step - loss: 9.2655 - mean_squared_error: 9.2655 - rmse: 3.0439 - mean_absolute_error: 2.4713 - val_loss: 6.0314 - val_mean_squared_error: 6.0314 - val_rmse: 2.4559 - val_mean_absolute_error: 1.9202 - lr: 0.0100
Epoch 52/1000
4/4 [==============================] - ETA: 0s - loss: 9.5512 - mean_squared_error: 9.5512 - rmse: 3.0905 - mean_absolute_error: 2.4748
Epoch 00052: val_loss did not improve from 6.03144
4/4 [==============================] - 1s 168ms/step - loss: 9.5512 - mean_squared_error: 9.5512 - rmse: 3.0905 - mean_absolute_error: 2.4748 - val_loss: 8.0463 - val_mean_squared_error: 8.0463 - val_rmse: 2.8366 - val_mean_absolute_error: 2.2314 - lr: 0.0100
Epoch 53/1000
4/4 [======================

4/4 [==============================] - ETA: 0s - loss: 7.7196 - mean_squared_error: 7.7196 - rmse: 2.7784 - mean_absolute_error: 2.1625
Epoch 00068: val_loss improved from 5.28489 to 5.16469, saving model to /tmp/lstm1v0.m5
4/4 [==============================] - 0s 120ms/step - loss: 7.7196 - mean_squared_error: 7.7196 - rmse: 2.7784 - mean_absolute_error: 2.1625 - val_loss: 5.1647 - val_mean_squared_error: 5.1647 - val_rmse: 2.2726 - val_mean_absolute_error: 1.7860 - lr: 0.0100
Epoch 69/1000
4/4 [==============================] - ETA: 0s - loss: 6.7737 - mean_squared_error: 6.7737 - rmse: 2.6026 - mean_absolute_error: 2.0264
Epoch 00069: val_loss did not improve from 5.16469
4/4 [==============================] - 0s 116ms/step - loss: 6.7737 - mean_squared_error: 6.7737 - rmse: 2.6026 - mean_absolute_error: 2.0264 - val_loss: 5.3155 - val_mean_squared_error: 5.3155 - val_rmse: 2.3055 - val_mean_absolute_error: 1.8193 - lr: 0.0100
Epoch 70/1000
4/4 [==============================] - ET

...

1/2 [==============>...............] - ETA: 0s - loss: 13.2441 - mean_squared_error: 13.2441 - rmse: 3.6392 - mean_absolute_error: 3.0110
Epoch 00137: val_loss did not improve from 15.34289
2/2 [==============================] - 0s 66ms/step - loss: 13.8908 - mean_squared_error: 13.8908 - rmse: 3.7270 - mean_absolute_error: 3.0764 - val_loss: 15.8402 - val_mean_squared_error: 15.8402 - val_rmse: 3.9800 - val_mean_absolute_error: 3.3973 - lr: 6.2500e-04
Epoch 138/1000
1/2 [==============>...............] - ETA: 0s - loss: 13.2154 - mean_squared_error: 13.2154 - rmse: 3.6353 - mean_absolute_error: 2.9878
Epoch 00138: val_loss did not improve from 15.34289
2/2 [==============================] - 0s 66ms/step - loss: 13.8894 - mean_squared_error: 13.8894 - rmse: 3.7269 - mean_absolute_error: 3.0757 - val_loss: 15.8416 - val_mean_squared_error: 15.8416 - val_rmse: 3.9801 - val_mean_absolute_error: 3.3972 - lr: 6.2500e-04
Epoch 139/1000
1/2 [==============>...............] - ETA: 0s - loss: 1

1/2 [==============>...............] - ETA: 0s - loss: 14.2305 - mean_squared_error: 14.2305 - rmse: 3.7723 - mean_absolute_error: 3.0795
Epoch 00154: val_loss did not improve from 15.34289
2/2 [==============================] - 0s 65ms/step - loss: 13.8811 - mean_squared_error: 13.8811 - rmse: 3.7257 - mean_absolute_error: 3.0801 - val_loss: 15.7948 - val_mean_squared_error: 15.7948 - val_rmse: 3.9743 - val_mean_absolute_error: 3.3971 - lr: 3.1250e-04
Epoch 155/1000
1/2 [==============>...............] - ETA: 0s - loss: 14.0791 - mean_squared_error: 14.0791 - rmse: 3.7522 - mean_absolute_error: 3.1099
Epoch 00155: val_loss did not improve from 15.34289
2/2 [==============================] - 0s 72ms/step - loss: 13.8800 - mean_squared_error: 13.8800 - rmse: 3.7256 - mean_absolute_error: 3.0792 - val_loss: 15.8031 - val_mean_squared_error: 15.8031 - val_rmse: 3.9753 - val_mean_absolute_error: 3.3968 - lr: 3.1250e-04
Epoch 156/1000
1/2 [==============>...............] - ETA: 0s - loss: 1

1/2 [==============>...............] - ETA: 0s - loss: 13.4598 - mean_squared_error: 13.4598 - rmse: 3.6688 - mean_absolute_error: 3.0614
Epoch 00171: val_loss did not improve from 15.34289
2/2 [==============================] - 0s 65ms/step - loss: 13.8730 - mean_squared_error: 13.8730 - rmse: 3.7247 - mean_absolute_error: 3.0742 - val_loss: 15.8235 - val_mean_squared_error: 15.8235 - val_rmse: 3.9779 - val_mean_absolute_error: 3.3956 - lr: 1.5625e-04
Epoch 172/1000
1/2 [==============>...............] - ETA: 0s - loss: 13.0931 - mean_squared_error: 13.0931 - rmse: 3.6184 - mean_absolute_error: 2.9894
Epoch 00172: val_loss did not improve from 15.34289
2/2 [==============================] - 0s 68ms/step - loss: 13.8722 - mean_squared_error: 13.8722 - rmse: 3.7245 - mean_absolute_error: 3.0735 - val_loss: 15.8264 - val_mean_squared_error: 15.8264 - val_rmse: 3.9782 - val_mean_absolute_error: 3.3955 - lr: 1.5625e-04
Epoch 173/1000
1/2 [==============>...............] - ETA: 0s - loss: 1

1/2 [==============>...............] - ETA: 0s - loss: 14.2892 - mean_squared_error: 14.2892 - rmse: 3.7801 - mean_absolute_error: 3.1405
Epoch 00188: val_loss did not improve from 15.34289
2/2 [==============================] - 0s 68ms/step - loss: 13.8684 - mean_squared_error: 13.8684 - rmse: 3.7240 - mean_absolute_error: 3.0721 - val_loss: 15.8274 - val_mean_squared_error: 15.8274 - val_rmse: 3.9784 - val_mean_absolute_error: 3.3950 - lr: 7.8125e-05
Epoch 189/1000
1/2 [==============>...............] - ETA: 0s - loss: 13.5929 - mean_squared_error: 13.5929 - rmse: 3.6869 - mean_absolute_error: 3.0459
Epoch 00189: val_loss did not improve from 15.34289
2/2 [==============================] - 0s 68ms/step - loss: 13.8682 - mean_squared_error: 13.8682 - rmse: 3.7240 - mean_absolute_error: 3.0720 - val_loss: 15.8277 - val_mean_squared_error: 15.8277 - val_rmse: 3.9784 - val_mean_absolute_error: 3.3950 - lr: 7.8125e-05
Epoch 190/1000
1/2 [==============>...............] - ETA: 0s - loss: 1

5/5 [==============================] - 0s 10ms/step - loss: 13.3215 - mean_squared_error: 13.3215 - rmse: 3.6499 - mean_absolute_error: 2.9602


In [60]:
df_lstm1v0 = (df_lstm1v01.merge(df_lstm1v02, how='outer')).merge(df_lstm1v03, how='outer')

### lstm1

In [61]:
df_lstm11, history_lstm11 = single_cross_site(lstm1, 'lstm1', '/tmp/lstm1.m5', 0.01, 0, 3, 4, X, y)
df_lstm12, history_lstm12 = single_cross_site(lstm1, 'lstm1', '/tmp/lstm1.m5', 0.01, 3, 0, 4, X, y)
df_lstm13, history_lstm13 = single_cross_site(lstm1, 'lstm1', '/tmp/lstm1.m5', 0.01, 4, 0, 3, X, y)

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 100, 72)]         0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               69200     
                                                                 
 dense_48 (Dense)            (None, 1)                 101       
                                                                 
Total params: 69,301
Trainable params: 69,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
4/4 [==============================] - ETA: 0s - loss: 58.6640 - mean_squared_error: 58.6640 - rmse: 7.6592 - mean_absolute_error: 6.2646  
Epoch 00001: val_loss improved from inf to 10.59812, saving model to /tmp/lstm1.m5
4/4 [==============================] - 2s 237ms/step - loss: 58.6640 - mean_square

Epoch 17/1000
4/4 [==============================] - ETA: 0s - loss: 12.6674 - mean_squared_error: 12.6674 - rmse: 3.5591 - mean_absolute_error: 2.8822
Epoch 00017: val_loss improved from 10.48051 to 10.46936, saving model to /tmp/lstm1.m5
4/4 [==============================] - 0s 126ms/step - loss: 12.6674 - mean_squared_error: 12.6674 - rmse: 3.5591 - mean_absolute_error: 2.8822 - val_loss: 10.4694 - val_mean_squared_error: 10.4694 - val_rmse: 3.2356 - val_mean_absolute_error: 2.7228 - lr: 0.0100
Epoch 18/1000
4/4 [==============================] - ETA: 0s - loss: 12.6501 - mean_squared_error: 12.6501 - rmse: 3.5567 - mean_absolute_error: 2.8991
Epoch 00018: val_loss did not improve from 10.46936
4/4 [==============================] - 0s 122ms/step - loss: 12.6501 - mean_squared_error: 12.6501 - rmse: 3.5567 - mean_absolute_error: 2.8991 - val_loss: 10.5435 - val_mean_squared_error: 10.5435 - val_rmse: 3.2471 - val_mean_absolute_error: 2.7514 - lr: 0.0100
Epoch 19/1000
4/4 [=========

Epoch 34/1000
4/4 [==============================] - ETA: 0s - loss: 12.4940 - mean_squared_error: 12.4940 - rmse: 3.5347 - mean_absolute_error: 2.9015
Epoch 00034: val_loss improved from 10.29676 to 10.24254, saving model to /tmp/lstm1.m5
4/4 [==============================] - 0s 117ms/step - loss: 12.4940 - mean_squared_error: 12.4940 - rmse: 3.5347 - mean_absolute_error: 2.9015 - val_loss: 10.2425 - val_mean_squared_error: 10.2425 - val_rmse: 3.2004 - val_mean_absolute_error: 2.7013 - lr: 0.0100
Epoch 35/1000
4/4 [==============================] - ETA: 0s - loss: 12.4517 - mean_squared_error: 12.4517 - rmse: 3.5287 - mean_absolute_error: 2.8594
Epoch 00035: val_loss improved from 10.24254 to 10.18983, saving model to /tmp/lstm1.m5
4/4 [==============================] - 0s 125ms/step - loss: 12.4517 - mean_squared_error: 12.4517 - rmse: 3.5287 - mean_absolute_error: 2.8594 - val_loss: 10.1898 - val_mean_squared_error: 10.1898 - val_rmse: 3.1922 - val_mean_absolute_error: 2.6903 - lr:

Epoch 51/1000
4/4 [==============================] - ETA: 0s - loss: 9.2404 - mean_squared_error: 9.2404 - rmse: 3.0398 - mean_absolute_error: 2.4275
Epoch 00051: val_loss did not improve from 7.15610
4/4 [==============================] - 0s 117ms/step - loss: 9.2404 - mean_squared_error: 9.2404 - rmse: 3.0398 - mean_absolute_error: 2.4275 - val_loss: 8.3727 - val_mean_squared_error: 8.3727 - val_rmse: 2.8936 - val_mean_absolute_error: 2.4347 - lr: 0.0100
Epoch 52/1000
4/4 [==============================] - ETA: 0s - loss: 10.3249 - mean_squared_error: 10.3249 - rmse: 3.2132 - mean_absolute_error: 2.5376
Epoch 00052: val_loss improved from 7.15610 to 6.99501, saving model to /tmp/lstm1.m5
4/4 [==============================] - 0s 118ms/step - loss: 10.3249 - mean_squared_error: 10.3249 - rmse: 3.2132 - mean_absolute_error: 2.5376 - val_loss: 6.9950 - val_mean_squared_error: 6.9950 - val_rmse: 2.6448 - val_mean_absolute_error: 2.2101 - lr: 0.0100
Epoch 53/1000
4/4 [====================

4/4 [==============================] - ETA: 0s - loss: 6.7912 - mean_squared_error: 6.7912 - rmse: 2.6060 - mean_absolute_error: 1.9869
Epoch 00068: val_loss did not improve from 5.09709
4/4 [==============================] - 0s 119ms/step - loss: 6.7912 - mean_squared_error: 6.7912 - rmse: 2.6060 - mean_absolute_error: 1.9869 - val_loss: 7.8740 - val_mean_squared_error: 7.8740 - val_rmse: 2.8061 - val_mean_absolute_error: 2.2769 - lr: 0.0100
Epoch 69/1000
4/4 [==============================] - ETA: 0s - loss: 6.6701 - mean_squared_error: 6.6701 - rmse: 2.5827 - mean_absolute_error: 2.0117
Epoch 00069: val_loss did not improve from 5.09709
4/4 [==============================] - 0s 115ms/step - loss: 6.6701 - mean_squared_error: 6.6701 - rmse: 2.5827 - mean_absolute_error: 2.0117 - val_loss: 5.3119 - val_mean_squared_error: 5.3119 - val_rmse: 2.3047 - val_mean_absolute_error: 1.9241 - lr: 0.0100
Epoch 70/1000
4/4 [==============================] - ETA: 0s - loss: 6.6957 - mean_squared_e

...

Epoch 32/1000
1/2 [==============>...............] - ETA: 0s - loss: 12.7692 - mean_squared_error: 12.7692 - rmse: 3.5734 - mean_absolute_error: 3.0854
Epoch 00032: val_loss did not improve from 13.47229
2/2 [==============================] - 0s 69ms/step - loss: 14.1286 - mean_squared_error: 14.1286 - rmse: 3.7588 - mean_absolute_error: 3.2199 - val_loss: 13.7343 - val_mean_squared_error: 13.7343 - val_rmse: 3.7060 - val_mean_absolute_error: 3.0503 - lr: 0.0050
Epoch 33/1000
1/2 [==============>...............] - ETA: 0s - loss: 13.3371 - mean_squared_error: 13.3371 - rmse: 3.6520 - mean_absolute_error: 3.1018
Epoch 00033: val_loss did not improve from 13.47229
2/2 [==============================] - 0s 67ms/step - loss: 14.1133 - mean_squared_error: 14.1133 - rmse: 3.7568 - mean_absolute_error: 3.2137 - val_loss: 13.8071 - val_mean_squared_error: 13.8071 - val_rmse: 3.7158 - val_mean_absolute_error: 3.0518 - lr: 0.0050
Epoch 34/1000
1/2 [==============>...............] - ETA: 0s - los

Epoch 50/1000
1/2 [==============>...............] - ETA: 0s - loss: 14.0407 - mean_squared_error: 14.0407 - rmse: 3.7471 - mean_absolute_error: 3.2200
Epoch 00050: val_loss did not improve from 13.47229
2/2 [==============================] - 0s 68ms/step - loss: 14.0716 - mean_squared_error: 14.0716 - rmse: 3.7512 - mean_absolute_error: 3.1940 - val_loss: 13.9938 - val_mean_squared_error: 13.9938 - val_rmse: 3.7408 - val_mean_absolute_error: 3.0547 - lr: 0.0025
Epoch 51/1000
1/2 [==============>...............] - ETA: 0s - loss: 14.7502 - mean_squared_error: 14.7502 - rmse: 3.8406 - mean_absolute_error: 3.2750
Epoch 00051: val_loss did not improve from 13.47229
2/2 [==============================] - 0s 66ms/step - loss: 14.0716 - mean_squared_error: 14.0716 - rmse: 3.7512 - mean_absolute_error: 3.1929 - val_loss: 14.0158 - val_mean_squared_error: 14.0158 - val_rmse: 3.7438 - val_mean_absolute_error: 3.0551 - lr: 0.0025
Epoch 52/1000
1/2 [==============>...............] - ETA: 0s - los

Epoch 68/1000
1/2 [==============>...............] - ETA: 0s - loss: 14.6171 - mean_squared_error: 14.6171 - rmse: 3.8232 - mean_absolute_error: 3.2537
Epoch 00068: val_loss did not improve from 13.47229
2/2 [==============================] - 0s 67ms/step - loss: 14.0706 - mean_squared_error: 14.0706 - rmse: 3.7511 - mean_absolute_error: 3.1880 - val_loss: 14.0626 - val_mean_squared_error: 14.0626 - val_rmse: 3.7500 - val_mean_absolute_error: 3.0554 - lr: 0.0012
Epoch 69/1000
1/2 [==============>...............] - ETA: 0s - loss: 14.5801 - mean_squared_error: 14.5801 - rmse: 3.8184 - mean_absolute_error: 3.2364
Epoch 00069: val_loss did not improve from 13.47229
2/2 [==============================] - 0s 66ms/step - loss: 14.0696 - mean_squared_error: 14.0696 - rmse: 3.7510 - mean_absolute_error: 3.1873 - val_loss: 14.0789 - val_mean_squared_error: 14.0789 - val_rmse: 3.7522 - val_mean_absolute_error: 3.0557 - lr: 0.0012
Epoch 70/1000
1/2 [==============>...............] - ETA: 0s - los

Epoch 86/1000
1/2 [==============>...............] - ETA: 0s - loss: 13.2434 - mean_squared_error: 13.2434 - rmse: 3.6392 - mean_absolute_error: 3.0846
Epoch 00086: val_loss did not improve from 13.47229

Epoch 00086: ReduceLROnPlateau reducing learning rate to 0.0006249999860301614.
2/2 [==============================] - 0s 73ms/step - loss: 14.0602 - mean_squared_error: 14.0602 - rmse: 3.7497 - mean_absolute_error: 3.1977 - val_loss: 13.8648 - val_mean_squared_error: 13.8648 - val_rmse: 3.7236 - val_mean_absolute_error: 3.0519 - lr: 0.0012
Epoch 87/1000
1/2 [==============>...............] - ETA: 0s - loss: 13.9620 - mean_squared_error: 13.9620 - rmse: 3.7366 - mean_absolute_error: 3.1793
Epoch 00087: val_loss did not improve from 13.47229
2/2 [==============================] - 0s 76ms/step - loss: 14.0621 - mean_squared_error: 14.0621 - rmse: 3.7499 - mean_absolute_error: 3.2001 - val_loss: 13.8404 - val_mean_squared_error: 13.8404 - val_rmse: 3.7203 - val_mean_absolute_error: 3.051

1/2 [==============>...............] - ETA: 0s - loss: 13.8567 - mean_squared_error: 13.8567 - rmse: 3.7225 - mean_absolute_error: 3.1962
Epoch 00103: val_loss did not improve from 13.47229
2/2 [==============================] - 0s 71ms/step - loss: 14.0601 - mean_squared_error: 14.0601 - rmse: 3.7497 - mean_absolute_error: 3.1968 - val_loss: 13.9135 - val_mean_squared_error: 13.9135 - val_rmse: 3.7301 - val_mean_absolute_error: 3.0526 - lr: 6.2500e-04
Epoch 104/1000
1/2 [==============>...............] - ETA: 0s - loss: 13.7263 - mean_squared_error: 13.7263 - rmse: 3.7049 - mean_absolute_error: 3.1696
Epoch 00104: val_loss did not improve from 13.47229
2/2 [==============================] - 0s 66ms/step - loss: 14.0562 - mean_squared_error: 14.0562 - rmse: 3.7492 - mean_absolute_error: 3.1959 - val_loss: 13.9055 - val_mean_squared_error: 13.9055 - val_rmse: 3.7290 - val_mean_absolute_error: 3.0524 - lr: 6.2500e-04
Epoch 105/1000
1/2 [==============>...............] - ETA: 0s - loss: 1

In [62]:
df_lstm1 = (df_lstm11.merge(df_lstm12, how='outer')).merge(df_lstm13, how='outer')

### lstm2

In [63]:
df_lstm21, history_lstm21 = single_cross_site(lstm2, 'lstm2', '/tmp/lstm2.m5', 0.01, 0, 3, 4, X, y)
df_lstm22, history_lstm22 = single_cross_site(lstm2, 'lstm2', '/tmp/lstm2.m5', 0.01, 3, 0, 4, X, y)
df_lstm23, history_lstm23 = single_cross_site(lstm2, 'lstm2', '/tmp/lstm2.m5', 0.01, 4, 0, 3, X, y)

Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 100, 72)]         0         
                                                                 
 lstm_6 (LSTM)               (None, 100, 100)          69200     
                                                                 
 lstm_7 (LSTM)               (None, 100)               80400     
                                                                 
 dense_51 (Dense)            (None, 1)                 101       
                                                                 
Total params: 149,701
Trainable params: 149,701
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
4/4 [==============================] - ETA: 0s - loss: 50.6140 - mean_squared_error: 50.6140 - rmse: 7.1144 - mean_absolute_error: 5.6448   
Epoch 00001: val_loss improved

4/4 [==============================] - ETA: 0s - loss: 12.4850 - mean_squared_error: 12.4850 - rmse: 3.5334 - mean_absolute_error: 2.8866
Epoch 00016: val_loss did not improve from 10.92411
4/4 [==============================] - 1s 244ms/step - loss: 12.4850 - mean_squared_error: 12.4850 - rmse: 3.5334 - mean_absolute_error: 2.8866 - val_loss: 10.9590 - val_mean_squared_error: 10.9590 - val_rmse: 3.3104 - val_mean_absolute_error: 2.7084 - lr: 0.0100
Epoch 17/1000
4/4 [==============================] - ETA: 0s - loss: 12.4808 - mean_squared_error: 12.4808 - rmse: 3.5328 - mean_absolute_error: 2.9022
Epoch 00017: val_loss did not improve from 10.92411
4/4 [==============================] - 1s 246ms/step - loss: 12.4808 - mean_squared_error: 12.4808 - rmse: 3.5328 - mean_absolute_error: 2.9022 - val_loss: 11.0066 - val_mean_squared_error: 11.0066 - val_rmse: 3.3176 - val_mean_absolute_error: 2.7258 - lr: 0.0100
Epoch 18/1000
4/4 [==============================] - ETA: 0s - loss: 12.5012 -

Epoch 34/1000
4/4 [==============================] - ETA: 0s - loss: 12.4930 - mean_squared_error: 12.4930 - rmse: 3.5345 - mean_absolute_error: 2.9138
Epoch 00034: val_loss did not improve from 10.92411
4/4 [==============================] - 1s 246ms/step - loss: 12.4930 - mean_squared_error: 12.4930 - rmse: 3.5345 - mean_absolute_error: 2.9138 - val_loss: 11.0124 - val_mean_squared_error: 11.0124 - val_rmse: 3.3185 - val_mean_absolute_error: 2.7276 - lr: 0.0050
Epoch 35/1000
4/4 [==============================] - ETA: 0s - loss: 12.4887 - mean_squared_error: 12.4887 - rmse: 3.5339 - mean_absolute_error: 2.9140
Epoch 00035: val_loss did not improve from 10.92411
4/4 [==============================] - 1s 249ms/step - loss: 12.4887 - mean_squared_error: 12.4887 - rmse: 3.5339 - mean_absolute_error: 2.9140 - val_loss: 10.9793 - val_mean_squared_error: 10.9793 - val_rmse: 3.3135 - val_mean_absolute_error: 2.7168 - lr: 0.0050
Epoch 36/1000
4/4 [==============================] - ETA: 0s - l

Epoch 52/1000
4/4 [==============================] - ETA: 0s - loss: 12.4925 - mean_squared_error: 12.4925 - rmse: 3.5345 - mean_absolute_error: 2.9102
Epoch 00052: val_loss did not improve from 10.92411
4/4 [==============================] - 1s 243ms/step - loss: 12.4925 - mean_squared_error: 12.4925 - rmse: 3.5345 - mean_absolute_error: 2.9102 - val_loss: 10.9658 - val_mean_squared_error: 10.9658 - val_rmse: 3.3115 - val_mean_absolute_error: 2.7115 - lr: 0.0025
Epoch 53/1000
4/4 [==============================] - ETA: 0s - loss: 12.4821 - mean_squared_error: 12.4821 - rmse: 3.5330 - mean_absolute_error: 2.8980
Epoch 00053: val_loss did not improve from 10.92411
4/4 [==============================] - 1s 248ms/step - loss: 12.4821 - mean_squared_error: 12.4821 - rmse: 3.5330 - mean_absolute_error: 2.8980 - val_loss: 10.9576 - val_mean_squared_error: 10.9576 - val_rmse: 3.3102 - val_mean_absolute_error: 2.7078 - lr: 0.0025
Epoch 54/1000
4/4 [==============================] - ETA: 0s - l

Epoch 70/1000
4/4 [==============================] - ETA: 0s - loss: 12.4846 - mean_squared_error: 12.4846 - rmse: 3.5334 - mean_absolute_error: 2.9049
Epoch 00070: val_loss did not improve from 10.92411
4/4 [==============================] - 1s 246ms/step - loss: 12.4846 - mean_squared_error: 12.4846 - rmse: 3.5334 - mean_absolute_error: 2.9049 - val_loss: 10.9690 - val_mean_squared_error: 10.9690 - val_rmse: 3.3119 - val_mean_absolute_error: 2.7131 - lr: 0.0012
Epoch 71/1000
4/4 [==============================] - ETA: 0s - loss: 12.4801 - mean_squared_error: 12.4801 - rmse: 3.5327 - mean_absolute_error: 2.9001
Epoch 00071: val_loss did not improve from 10.92411
4/4 [==============================] - 1s 245ms/step - loss: 12.4801 - mean_squared_error: 12.4801 - rmse: 3.5327 - mean_absolute_error: 2.9001 - val_loss: 10.9618 - val_mean_squared_error: 10.9618 - val_rmse: 3.3109 - val_mean_absolute_error: 2.7101 - lr: 0.0012
Epoch 72/1000
4/4 [==============================] - ETA: 0s - l

...

Epoch 26/1000
2/2 [==============================] - ETA: 0s - loss: 15.2090 - mean_squared_error: 15.2090 - rmse: 3.8999 - mean_absolute_error: 3.3443
Epoch 00026: val_loss did not improve from 11.76253
2/2 [==============================] - 0s 120ms/step - loss: 15.2090 - mean_squared_error: 15.2090 - rmse: 3.8999 - mean_absolute_error: 3.3443 - val_loss: 12.9357 - val_mean_squared_error: 12.9357 - val_rmse: 3.5966 - val_mean_absolute_error: 3.1195 - lr: 0.0100
Epoch 27/1000
2/2 [==============================] - ETA: 0s - loss: 15.2689 - mean_squared_error: 15.2689 - rmse: 3.9075 - mean_absolute_error: 3.3630
Epoch 00027: val_loss did not improve from 11.76253
2/2 [==============================] - 0s 119ms/step - loss: 15.2689 - mean_squared_error: 15.2689 - rmse: 3.9075 - mean_absolute_error: 3.3630 - val_loss: 13.1426 - val_mean_squared_error: 13.1426 - val_rmse: 3.6253 - val_mean_absolute_error: 3.1573 - lr: 0.0100
Epoch 28/1000
2/2 [==============================] - ETA: 0s - l

Epoch 44/1000
2/2 [==============================] - ETA: 0s - loss: 15.1962 - mean_squared_error: 15.1962 - rmse: 3.8982 - mean_absolute_error: 3.3104
Epoch 00044: val_loss did not improve from 11.76253
2/2 [==============================] - 0s 116ms/step - loss: 15.1962 - mean_squared_error: 15.1962 - rmse: 3.8982 - mean_absolute_error: 3.3104 - val_loss: 12.1275 - val_mean_squared_error: 12.1275 - val_rmse: 3.4825 - val_mean_absolute_error: 2.9629 - lr: 0.0050
Epoch 45/1000
2/2 [==============================] - ETA: 0s - loss: 15.2138 - mean_squared_error: 15.2138 - rmse: 3.9005 - mean_absolute_error: 3.3055
Epoch 00045: val_loss did not improve from 11.76253
2/2 [==============================] - 0s 115ms/step - loss: 15.2138 - mean_squared_error: 15.2138 - rmse: 3.9005 - mean_absolute_error: 3.3055 - val_loss: 12.0603 - val_mean_squared_error: 12.0603 - val_rmse: 3.4728 - val_mean_absolute_error: 2.9475 - lr: 0.0050
Epoch 46/1000
2/2 [==============================] - ETA: 0s - l

Epoch 62/1000
2/2 [==============================] - ETA: 0s - loss: 15.2146 - mean_squared_error: 15.2146 - rmse: 3.9006 - mean_absolute_error: 3.3446
Epoch 00062: val_loss did not improve from 11.76253
2/2 [==============================] - 0s 117ms/step - loss: 15.2146 - mean_squared_error: 15.2146 - rmse: 3.9006 - mean_absolute_error: 3.3446 - val_loss: 12.7488 - val_mean_squared_error: 12.7488 - val_rmse: 3.5705 - val_mean_absolute_error: 3.0834 - lr: 0.0025
Epoch 63/1000
2/2 [==============================] - ETA: 0s - loss: 15.2327 - mean_squared_error: 15.2327 - rmse: 3.9029 - mean_absolute_error: 3.3507
Epoch 00063: val_loss did not improve from 11.76253
2/2 [==============================] - 0s 117ms/step - loss: 15.2327 - mean_squared_error: 15.2327 - rmse: 3.9029 - mean_absolute_error: 3.3507 - val_loss: 12.7877 - val_mean_squared_error: 12.7877 - val_rmse: 3.5760 - val_mean_absolute_error: 3.0904 - lr: 0.0025
Epoch 64/1000
2/2 [==============================] - ETA: 0s - l

Epoch 80/1000
2/2 [==============================] - ETA: 0s - loss: 15.1867 - mean_squared_error: 15.1867 - rmse: 3.8970 - mean_absolute_error: 3.3247
Epoch 00080: val_loss did not improve from 11.76253
2/2 [==============================] - 0s 114ms/step - loss: 15.1867 - mean_squared_error: 15.1867 - rmse: 3.8970 - mean_absolute_error: 3.3247 - val_loss: 12.4057 - val_mean_squared_error: 12.4057 - val_rmse: 3.5222 - val_mean_absolute_error: 3.0175 - lr: 0.0012
Epoch 81/1000
2/2 [==============================] - ETA: 0s - loss: 15.1866 - mean_squared_error: 15.1866 - rmse: 3.8970 - mean_absolute_error: 3.3250
Epoch 00081: val_loss did not improve from 11.76253
2/2 [==============================] - 0s 117ms/step - loss: 15.1866 - mean_squared_error: 15.1866 - rmse: 3.8970 - mean_absolute_error: 3.3250 - val_loss: 12.4012 - val_mean_squared_error: 12.4012 - val_rmse: 3.5215 - val_mean_absolute_error: 3.0166 - lr: 0.0012
Epoch 82/1000
2/2 [==============================] - ETA: 0s - l

2/2 [==============================] - ETA: 0s - loss: 15.1860 - mean_squared_error: 15.1860 - rmse: 3.8969 - mean_absolute_error: 3.3224
Epoch 00097: val_loss did not improve from 11.76253
2/2 [==============================] - 0s 146ms/step - loss: 15.1860 - mean_squared_error: 15.1860 - rmse: 3.8969 - mean_absolute_error: 3.3224 - val_loss: 12.3787 - val_mean_squared_error: 12.3787 - val_rmse: 3.5183 - val_mean_absolute_error: 3.0122 - lr: 6.2500e-04
Epoch 98/1000
2/2 [==============================] - ETA: 0s - loss: 15.1854 - mean_squared_error: 15.1854 - rmse: 3.8968 - mean_absolute_error: 3.3232
Epoch 00098: val_loss did not improve from 11.76253
2/2 [==============================] - 0s 119ms/step - loss: 15.1854 - mean_squared_error: 15.1854 - rmse: 3.8968 - mean_absolute_error: 3.3232 - val_loss: 12.3952 - val_mean_squared_error: 12.3952 - val_rmse: 3.5207 - val_mean_absolute_error: 3.0154 - lr: 6.2500e-04
Epoch 99/1000
2/2 [==============================] - ETA: 0s - loss: 1

In [64]:
df_lstm2 = (df_lstm21.merge(df_lstm22, how='outer')).merge(df_lstm23, how='outer')

### blstm1

In [65]:
df_blstm11, history_blstm11 = single_cross_site(blstm1, 'blstm1', '/tmp/blstm1.m5', 0.01, 0, 3, 4, X, y)
df_blstm12, history_blstm12 = single_cross_site(blstm1, 'blstm1', '/tmp/blstm1.m5', 0.01, 3, 0, 4, X, y)
df_blstm13, history_blstm13 = single_cross_site(blstm1, 'blstm1', '/tmp/blstm1.m5', 0.01, 4, 0, 3, X, y)

Model: "model_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 100, 72)]         0         
                                                                 
 bidirectional (Bidirectiona  (None, 200)              138400    
 l)                                                              
                                                                 
 dense_54 (Dense)            (None, 1)                 201       
                                                                 
Total params: 138,601
Trainable params: 138,601
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
4/4 [==============================] - ETA: 0s - loss: 51.1575 - mean_squared_error: 51.1575 - rmse: 7.1524 - mean_absolute_error: 5.5129  
Epoch 00001: val_loss improved from inf to 16.48779, saving model to /tmp/blstm1.m5
4/4 [========

Epoch 16/1000
4/4 [==============================] - ETA: 0s - loss: 11.9411 - mean_squared_error: 11.9411 - rmse: 3.4556 - mean_absolute_error: 2.7974
Epoch 00016: val_loss improved from 12.01123 to 11.98821, saving model to /tmp/blstm1.m5
4/4 [==============================] - 1s 196ms/step - loss: 11.9411 - mean_squared_error: 11.9411 - rmse: 3.4556 - mean_absolute_error: 2.7974 - val_loss: 11.9882 - val_mean_squared_error: 11.9882 - val_rmse: 3.4624 - val_mean_absolute_error: 2.7729 - lr: 0.0100
Epoch 17/1000
4/4 [==============================] - ETA: 0s - loss: 11.9220 - mean_squared_error: 11.9220 - rmse: 3.4528 - mean_absolute_error: 2.7955
Epoch 00017: val_loss improved from 11.98821 to 11.96705, saving model to /tmp/blstm1.m5
4/4 [==============================] - 1s 199ms/step - loss: 11.9220 - mean_squared_error: 11.9220 - rmse: 3.4528 - mean_absolute_error: 2.7955 - val_loss: 11.9671 - val_mean_squared_error: 11.9671 - val_rmse: 3.4593 - val_mean_absolute_error: 2.7780 - l

Epoch 33/1000
4/4 [==============================] - ETA: 0s - loss: 10.7897 - mean_squared_error: 10.7897 - rmse: 3.2848 - mean_absolute_error: 2.7085
Epoch 00033: val_loss did not improve from 9.85093
4/4 [==============================] - 1s 178ms/step - loss: 10.7897 - mean_squared_error: 10.7897 - rmse: 3.2848 - mean_absolute_error: 2.7085 - val_loss: 10.1826 - val_mean_squared_error: 10.1826 - val_rmse: 3.1910 - val_mean_absolute_error: 2.6061 - lr: 0.0100
Epoch 34/1000
4/4 [==============================] - ETA: 0s - loss: 10.3577 - mean_squared_error: 10.3577 - rmse: 3.2183 - mean_absolute_error: 2.5763
Epoch 00034: val_loss did not improve from 9.85093
4/4 [==============================] - 1s 179ms/step - loss: 10.3577 - mean_squared_error: 10.3577 - rmse: 3.2183 - mean_absolute_error: 2.5763 - val_loss: 10.8098 - val_mean_squared_error: 10.8098 - val_rmse: 3.2878 - val_mean_absolute_error: 2.7398 - lr: 0.0100
Epoch 35/1000
4/4 [==============================] - ETA: 0s - los

4/4 [==============================] - ETA: 0s - loss: 7.6903 - mean_squared_error: 7.6903 - rmse: 2.7731 - mean_absolute_error: 2.1746
Epoch 00050: val_loss improved from 6.98872 to 6.37363, saving model to /tmp/blstm1.m5
4/4 [==============================] - 1s 185ms/step - loss: 7.6903 - mean_squared_error: 7.6903 - rmse: 2.7731 - mean_absolute_error: 2.1746 - val_loss: 6.3736 - val_mean_squared_error: 6.3736 - val_rmse: 2.5246 - val_mean_absolute_error: 2.0786 - lr: 0.0100
Epoch 51/1000
4/4 [==============================] - ETA: 0s - loss: 7.3452 - mean_squared_error: 7.3452 - rmse: 2.7102 - mean_absolute_error: 2.1305
Epoch 00051: val_loss did not improve from 6.37363
4/4 [==============================] - 1s 180ms/step - loss: 7.3452 - mean_squared_error: 7.3452 - rmse: 2.7102 - mean_absolute_error: 2.1305 - val_loss: 6.9828 - val_mean_squared_error: 6.9828 - val_rmse: 2.6425 - val_mean_absolute_error: 2.1717 - lr: 0.0100
Epoch 52/1000
4/4 [==============================] - ETA

4/4 [==============================] - ETA: 0s - loss: 6.2572 - mean_squared_error: 6.2572 - rmse: 2.5014 - mean_absolute_error: 1.9642
Epoch 00067: val_loss did not improve from 5.72624
4/4 [==============================] - 1s 177ms/step - loss: 6.2572 - mean_squared_error: 6.2572 - rmse: 2.5014 - mean_absolute_error: 1.9642 - val_loss: 6.2197 - val_mean_squared_error: 6.2197 - val_rmse: 2.4939 - val_mean_absolute_error: 2.0495 - lr: 0.0100
Epoch 68/1000
4/4 [==============================] - ETA: 0s - loss: 6.6971 - mean_squared_error: 6.6971 - rmse: 2.5879 - mean_absolute_error: 2.0322
Epoch 00068: val_loss did not improve from 5.72624
4/4 [==============================] - 1s 175ms/step - loss: 6.6971 - mean_squared_error: 6.6971 - rmse: 2.5879 - mean_absolute_error: 2.0322 - val_loss: 7.1999 - val_mean_squared_error: 7.1999 - val_rmse: 2.6833 - val_mean_absolute_error: 2.1364 - lr: 0.0100
Epoch 69/1000
4/4 [==============================] - ETA: 0s - loss: 7.2339 - mean_squared_e

...

Epoch 28/1000
2/2 [==============================] - ETA: 0s - loss: 15.0301 - mean_squared_error: 15.0301 - rmse: 3.8769 - mean_absolute_error: 3.2199
Epoch 00028: val_loss did not improve from 11.74772
2/2 [==============================] - 0s 89ms/step - loss: 15.0301 - mean_squared_error: 15.0301 - rmse: 3.8769 - mean_absolute_error: 3.2199 - val_loss: 12.1459 - val_mean_squared_error: 12.1459 - val_rmse: 3.4851 - val_mean_absolute_error: 3.0370 - lr: 0.0100
Epoch 29/1000
1/2 [==============>...............] - ETA: 0s - loss: 14.6017 - mean_squared_error: 14.6017 - rmse: 3.8212 - mean_absolute_error: 3.1911
Epoch 00029: val_loss did not improve from 11.74772

Epoch 00029: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.
2/2 [==============================] - 0s 83ms/step - loss: 15.0078 - mean_squared_error: 15.0078 - rmse: 3.8740 - mean_absolute_error: 3.2186 - val_loss: 12.3848 - val_mean_squared_error: 12.3848 - val_rmse: 3.5192 - val_mean_absolute_error: 3.0702

Epoch 46/1000
1/2 [==============>...............] - ETA: 0s - loss: 14.5370 - mean_squared_error: 14.5370 - rmse: 3.8127 - mean_absolute_error: 3.1795
Epoch 00046: val_loss did not improve from 11.74772
2/2 [==============================] - 0s 82ms/step - loss: 14.8038 - mean_squared_error: 14.8038 - rmse: 3.8476 - mean_absolute_error: 3.2315 - val_loss: 12.5042 - val_mean_squared_error: 12.5042 - val_rmse: 3.5361 - val_mean_absolute_error: 3.0832 - lr: 0.0050
Epoch 47/1000
2/2 [==============================] - ETA: 0s - loss: 14.8002 - mean_squared_error: 14.8002 - rmse: 3.8471 - mean_absolute_error: 3.2327
Epoch 00047: val_loss did not improve from 11.74772
2/2 [==============================] - 0s 134ms/step - loss: 14.8002 - mean_squared_error: 14.8002 - rmse: 3.8471 - mean_absolute_error: 3.2327 - val_loss: 12.4968 - val_mean_squared_error: 12.4968 - val_rmse: 3.5351 - val_mean_absolute_error: 3.0821 - lr: 0.0050
Epoch 48/1000
1/2 [==============>...............] - ETA: 0s - lo

Epoch 64/1000
1/2 [==============>...............] - ETA: 0s - loss: 15.4665 - mean_squared_error: 15.4665 - rmse: 3.9328 - mean_absolute_error: 3.2986
Epoch 00064: val_loss did not improve from 11.74772
2/2 [==============================] - 0s 83ms/step - loss: 14.6510 - mean_squared_error: 14.6510 - rmse: 3.8277 - mean_absolute_error: 3.2233 - val_loss: 12.6639 - val_mean_squared_error: 12.6639 - val_rmse: 3.5586 - val_mean_absolute_error: 3.0982 - lr: 0.0025
Epoch 65/1000
1/2 [==============>...............] - ETA: 0s - loss: 14.5715 - mean_squared_error: 14.5715 - rmse: 3.8173 - mean_absolute_error: 3.2256
Epoch 00065: val_loss did not improve from 11.74772
2/2 [==============================] - 0s 83ms/step - loss: 14.6434 - mean_squared_error: 14.6434 - rmse: 3.8267 - mean_absolute_error: 3.2266 - val_loss: 12.7436 - val_mean_squared_error: 12.7436 - val_rmse: 3.5698 - val_mean_absolute_error: 3.1077 - lr: 0.0025
Epoch 66/1000
1/2 [==============>...............] - ETA: 0s - los

Epoch 82/1000
1/2 [==============>...............] - ETA: 0s - loss: 14.3025 - mean_squared_error: 14.3025 - rmse: 3.7819 - mean_absolute_error: 3.2173
Epoch 00082: val_loss did not improve from 11.74772
2/2 [==============================] - 0s 82ms/step - loss: 14.5182 - mean_squared_error: 14.5182 - rmse: 3.8103 - mean_absolute_error: 3.1914 - val_loss: 12.1320 - val_mean_squared_error: 12.1320 - val_rmse: 3.4831 - val_mean_absolute_error: 3.0310 - lr: 0.0012
Epoch 83/1000
1/2 [==============>...............] - ETA: 0s - loss: 13.5002 - mean_squared_error: 13.5002 - rmse: 3.6743 - mean_absolute_error: 3.1004
Epoch 00083: val_loss did not improve from 11.74772
2/2 [==============================] - 0s 81ms/step - loss: 14.5128 - mean_squared_error: 14.5128 - rmse: 3.8096 - mean_absolute_error: 3.1893 - val_loss: 12.1627 - val_mean_squared_error: 12.1627 - val_rmse: 3.4875 - val_mean_absolute_error: 3.0347 - lr: 0.0012
Epoch 84/1000
1/2 [==============>...............] - ETA: 0s - los

1/2 [==============>...............] - ETA: 0s - loss: 13.4129 - mean_squared_error: 13.4129 - rmse: 3.6624 - mean_absolute_error: 3.1270
Epoch 00099: val_loss did not improve from 11.74772
2/2 [==============================] - 0s 81ms/step - loss: 14.4272 - mean_squared_error: 14.4272 - rmse: 3.7983 - mean_absolute_error: 3.2008 - val_loss: 12.4792 - val_mean_squared_error: 12.4792 - val_rmse: 3.5326 - val_mean_absolute_error: 3.0707 - lr: 6.2500e-04
Epoch 100/1000
1/2 [==============>...............] - ETA: 0s - loss: 15.0525 - mean_squared_error: 15.0525 - rmse: 3.8798 - mean_absolute_error: 3.2401
Epoch 00100: val_loss did not improve from 11.74772
2/2 [==============================] - 0s 81ms/step - loss: 14.4173 - mean_squared_error: 14.4173 - rmse: 3.7970 - mean_absolute_error: 3.1988 - val_loss: 12.4355 - val_mean_squared_error: 12.4355 - val_rmse: 3.5264 - val_mean_absolute_error: 3.0651 - lr: 6.2500e-04
Epoch 101/1000
1/2 [==============>...............] - ETA: 0s - loss: 1

In [66]:
df_blstm1 = (df_blstm11.merge(df_blstm12, how='outer')).merge(df_blstm13, how='outer')

### blstm2

In [67]:
df_blstm21, history_blstm21 = single_cross_site(blstm2, 'blstm2', '/tmp/blstm2.m5', 0.01, 0, 3, 4, X, y)
df_blstm22, history_blstm22 = single_cross_site(blstm2, 'blstm2', '/tmp/blstm2.m5', 0.01, 3, 0, 4, X, y)
df_blstm23, history_blstm23 = single_cross_site(blstm2, 'blstm2', '/tmp/blstm2.m5', 0.01, 4, 0, 3, X, y)

Model: "model_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_29 (InputLayer)       [(None, 100, 72)]         0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 100, 200)         138400    
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, 200)              240800    
 nal)                                                            
                                                                 
 dense_57 (Dense)            (None, 1)                 201       
                                                                 
Total params: 379,401
Trainable params: 379,401
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
4/4 [==============================] - 

Epoch 16/1000
4/4 [==============================] - ETA: 0s - loss: 12.4236 - mean_squared_error: 12.4236 - rmse: 3.5247 - mean_absolute_error: 2.8980
Epoch 00016: val_loss did not improve from 11.36669
4/4 [==============================] - 2s 461ms/step - loss: 12.4236 - mean_squared_error: 12.4236 - rmse: 3.5247 - mean_absolute_error: 2.8980 - val_loss: 11.5460 - val_mean_squared_error: 11.5460 - val_rmse: 3.3979 - val_mean_absolute_error: 2.7726 - lr: 0.0100
Epoch 17/1000
4/4 [==============================] - ETA: 0s - loss: 12.4349 - mean_squared_error: 12.4349 - rmse: 3.5263 - mean_absolute_error: 2.9045
Epoch 00017: val_loss did not improve from 11.36669
4/4 [==============================] - 2s 456ms/step - loss: 12.4349 - mean_squared_error: 12.4349 - rmse: 3.5263 - mean_absolute_error: 2.9045 - val_loss: 11.6580 - val_mean_squared_error: 11.6580 - val_rmse: 3.4144 - val_mean_absolute_error: 2.8037 - lr: 0.0100
Epoch 18/1000
4/4 [==============================] - ETA: 0s - l

4/4 [==============================] - ETA: 0s - loss: 11.6727 - mean_squared_error: 11.6727 - rmse: 3.4165 - mean_absolute_error: 2.8116
Epoch 00033: val_loss improved from 11.01186 to 10.54569, saving model to /tmp/blstm2.m5
4/4 [==============================] - 2s 475ms/step - loss: 11.6727 - mean_squared_error: 11.6727 - rmse: 3.4165 - mean_absolute_error: 2.8116 - val_loss: 10.5457 - val_mean_squared_error: 10.5457 - val_rmse: 3.2474 - val_mean_absolute_error: 2.6155 - lr: 0.0050
Epoch 34/1000
4/4 [==============================] - ETA: 0s - loss: 11.4971 - mean_squared_error: 11.4971 - rmse: 3.3907 - mean_absolute_error: 2.7538
Epoch 00034: val_loss did not improve from 10.54569
4/4 [==============================] - 2s 466ms/step - loss: 11.4971 - mean_squared_error: 11.4971 - rmse: 3.3907 - mean_absolute_error: 2.7538 - val_loss: 10.8900 - val_mean_squared_error: 10.8900 - val_rmse: 3.3000 - val_mean_absolute_error: 2.6995 - lr: 0.0050
Epoch 35/1000
4/4 [======================

Epoch 50/1000
4/4 [==============================] - ETA: 0s - loss: 8.2658 - mean_squared_error: 8.2658 - rmse: 2.8750 - mean_absolute_error: 2.3003
Epoch 00050: val_loss did not improve from 7.85033
4/4 [==============================] - 2s 465ms/step - loss: 8.2658 - mean_squared_error: 8.2658 - rmse: 2.8750 - mean_absolute_error: 2.3003 - val_loss: 7.9978 - val_mean_squared_error: 7.9978 - val_rmse: 2.8280 - val_mean_absolute_error: 2.2076 - lr: 0.0050
Epoch 51/1000
4/4 [==============================] - ETA: 0s - loss: 7.8867 - mean_squared_error: 7.8867 - rmse: 2.8083 - mean_absolute_error: 2.2663
Epoch 00051: val_loss improved from 7.85033 to 7.83532, saving model to /tmp/blstm2.m5
4/4 [==============================] - 2s 491ms/step - loss: 7.8867 - mean_squared_error: 7.8867 - rmse: 2.8083 - mean_absolute_error: 2.2663 - val_loss: 7.8353 - val_mean_squared_error: 7.8353 - val_rmse: 2.7992 - val_mean_absolute_error: 2.1780 - lr: 0.0050
Epoch 52/1000
4/4 [=======================

Epoch 68/1000
4/4 [==============================] - ETA: 0s - loss: 7.3299 - mean_squared_error: 7.3299 - rmse: 2.7074 - mean_absolute_error: 2.1494
Epoch 00068: val_loss did not improve from 7.12914
4/4 [==============================] - 2s 460ms/step - loss: 7.3299 - mean_squared_error: 7.3299 - rmse: 2.7074 - mean_absolute_error: 2.1494 - val_loss: 7.2579 - val_mean_squared_error: 7.2579 - val_rmse: 2.6940 - val_mean_absolute_error: 2.0800 - lr: 0.0050
Epoch 69/1000
4/4 [==============================] - ETA: 0s - loss: 7.2507 - mean_squared_error: 7.2507 - rmse: 2.6927 - mean_absolute_error: 2.1203
Epoch 00069: val_loss did not improve from 7.12914
4/4 [==============================] - 2s 466ms/step - loss: 7.2507 - mean_squared_error: 7.2507 - rmse: 2.6927 - mean_absolute_error: 2.1203 - val_loss: 7.1696 - val_mean_squared_error: 7.1696 - val_rmse: 2.6776 - val_mean_absolute_error: 2.1394 - lr: 0.0050
Epoch 70/1000
4/4 [==============================] - ETA: 0s - loss: 6.7465 - 

...

Epoch 231/1000
2/2 [==============================] - ETA: 0s - loss: 6.3872 - mean_squared_error: 6.3872 - rmse: 2.5273 - mean_absolute_error: 1.9688
Epoch 00231: val_loss did not improve from 4.83926
2/2 [==============================] - 1s 171ms/step - loss: 6.3872 - mean_squared_error: 6.3872 - rmse: 2.5273 - mean_absolute_error: 1.9688 - val_loss: 5.0932 - val_mean_squared_error: 5.0932 - val_rmse: 2.2568 - val_mean_absolute_error: 1.7694 - lr: 0.0025
Epoch 232/1000
2/2 [==============================] - ETA: 0s - loss: 5.8417 - mean_squared_error: 5.8417 - rmse: 2.4170 - mean_absolute_error: 1.9181
Epoch 00232: val_loss did not improve from 4.83926
2/2 [==============================] - 1s 168ms/step - loss: 5.8417 - mean_squared_error: 5.8417 - rmse: 2.4170 - mean_absolute_error: 1.9181 - val_loss: 6.0896 - val_mean_squared_error: 6.0896 - val_rmse: 2.4677 - val_mean_absolute_error: 1.9675 - lr: 0.0025
Epoch 233/1000
2/2 [==============================] - ETA: 0s - loss: 6.4683

Epoch 249/1000
2/2 [==============================] - ETA: 0s - loss: 5.9060 - mean_squared_error: 5.9060 - rmse: 2.4302 - mean_absolute_error: 1.9425
Epoch 00249: val_loss did not improve from 4.83926
2/2 [==============================] - 1s 176ms/step - loss: 5.9060 - mean_squared_error: 5.9060 - rmse: 2.4302 - mean_absolute_error: 1.9425 - val_loss: 5.4181 - val_mean_squared_error: 5.4181 - val_rmse: 2.3277 - val_mean_absolute_error: 1.7991 - lr: 0.0012
Epoch 250/1000
2/2 [==============================] - ETA: 0s - loss: 5.8469 - mean_squared_error: 5.8469 - rmse: 2.4180 - mean_absolute_error: 1.9242
Epoch 00250: val_loss did not improve from 4.83926
2/2 [==============================] - 1s 176ms/step - loss: 5.8469 - mean_squared_error: 5.8469 - rmse: 2.4180 - mean_absolute_error: 1.9242 - val_loss: 5.1409 - val_mean_squared_error: 5.1409 - val_rmse: 2.2673 - val_mean_absolute_error: 1.7662 - lr: 0.0012
Epoch 251/1000
2/2 [==============================] - ETA: 0s - loss: 5.7945

Epoch 267/1000
2/2 [==============================] - ETA: 0s - loss: 5.6851 - mean_squared_error: 5.6851 - rmse: 2.3843 - mean_absolute_error: 1.8828
Epoch 00267: val_loss did not improve from 4.83926
2/2 [==============================] - 1s 176ms/step - loss: 5.6851 - mean_squared_error: 5.6851 - rmse: 2.3843 - mean_absolute_error: 1.8828 - val_loss: 4.9320 - val_mean_squared_error: 4.9320 - val_rmse: 2.2208 - val_mean_absolute_error: 1.7480 - lr: 6.2500e-04
Epoch 268/1000
2/2 [==============================] - ETA: 0s - loss: 5.6921 - mean_squared_error: 5.6921 - rmse: 2.3858 - mean_absolute_error: 1.8855
Epoch 00268: val_loss did not improve from 4.83926
2/2 [==============================] - 1s 170ms/step - loss: 5.6921 - mean_squared_error: 5.6921 - rmse: 2.3858 - mean_absolute_error: 1.8855 - val_loss: 4.9726 - val_mean_squared_error: 4.9726 - val_rmse: 2.2299 - val_mean_absolute_error: 1.7473 - lr: 6.2500e-04
Epoch 269/1000
2/2 [==============================] - ETA: 0s - loss

Epoch 285/1000
2/2 [==============================] - ETA: 0s - loss: 5.7266 - mean_squared_error: 5.7266 - rmse: 2.3930 - mean_absolute_error: 1.8958
Epoch 00285: val_loss did not improve from 4.83926
2/2 [==============================] - 1s 172ms/step - loss: 5.7266 - mean_squared_error: 5.7266 - rmse: 2.3930 - mean_absolute_error: 1.8958 - val_loss: 5.2844 - val_mean_squared_error: 5.2844 - val_rmse: 2.2988 - val_mean_absolute_error: 1.7706 - lr: 3.1250e-04
Epoch 286/1000
2/2 [==============================] - ETA: 0s - loss: 5.7398 - mean_squared_error: 5.7398 - rmse: 2.3958 - mean_absolute_error: 1.8962
Epoch 00286: val_loss did not improve from 4.83926
2/2 [==============================] - 1s 176ms/step - loss: 5.7398 - mean_squared_error: 5.7398 - rmse: 2.3958 - mean_absolute_error: 1.8962 - val_loss: 5.2144 - val_mean_squared_error: 5.2144 - val_rmse: 2.2835 - val_mean_absolute_error: 1.7578 - lr: 3.1250e-04
Epoch 287/1000
2/2 [==============================] - ETA: 0s - loss

Epoch 303/1000
2/2 [==============================] - ETA: 0s - loss: 5.6576 - mean_squared_error: 5.6576 - rmse: 2.3786 - mean_absolute_error: 1.8564
Epoch 00303: val_loss did not improve from 4.83926
2/2 [==============================] - 1s 175ms/step - loss: 5.6576 - mean_squared_error: 5.6576 - rmse: 2.3786 - mean_absolute_error: 1.8564 - val_loss: 4.9996 - val_mean_squared_error: 4.9996 - val_rmse: 2.2360 - val_mean_absolute_error: 1.7410 - lr: 3.1250e-04
Epoch 304/1000
2/2 [==============================] - ETA: 0s - loss: 5.6265 - mean_squared_error: 5.6265 - rmse: 2.3720 - mean_absolute_error: 1.8512
Epoch 00304: val_loss did not improve from 4.83926

Epoch 00304: ReduceLROnPlateau reducing learning rate to 0.00015624999650754035.
5/5 [==============================] - 0s 36ms/step - loss: 14.5944 - mean_squared_error: 14.5944 - rmse: 3.8203 - mean_absolute_error: 2.8983


In [68]:
df_blstm2 = (df_blstm21.merge(df_blstm22, how='outer')).merge(df_blstm23, how='outer')

### lstm_fcn

In [71]:
df_fcn1, history_fcn1 = single_cross_site(lstm_fcn, 'lstm_fcn', '/tmp/fcn.m5', 0.01, 0, 3, 4, X, y)
df_fcn2, history_fcn2 = single_cross_site(lstm_fcn, 'lstm_fcn', '/tmp/fcn.m5', 0.01, 3, 0, 4, X, y)
df_fcn3, history_fcn3 = single_cross_site(lstm_fcn, 'lstm_fcn', '/tmp/fcn.m5', 0.01, 4, 0, 3, X, y)

Model: "model_34"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_35 (InputLayer)          [(None, 100, 72)]    0           []                               
                                                                                                  
 conv1d_100 (Conv1D)            (None, 100, 128)     73856       ['input_35[0][0]']               
                                                                                                  
 batch_normalization_45 (BatchN  (None, 100, 128)    512         ['conv1d_100[0][0]']             
 ormalization)                                                                                    
                                                                                                  
 activation_45 (Activation)     (None, 100, 128)     0           ['batch_normalization_45[0

Epoch 9/1000
4/4 [==============================] - ETA: 0s - loss: 5.1027 - mean_squared_error: 5.1027 - rmse: 2.2589 - mean_absolute_error: 1.8049
Epoch 00009: val_loss did not improve from 40609.30859
4/4 [==============================] - 1s 244ms/step - loss: 5.1027 - mean_squared_error: 5.1027 - rmse: 2.2589 - mean_absolute_error: 1.8049 - val_loss: 49036.9531 - val_mean_squared_error: 49036.9531 - val_rmse: 221.4429 - val_mean_absolute_error: 219.8163 - lr: 0.0100
Epoch 10/1000
4/4 [==============================] - ETA: 0s - loss: 5.4342 - mean_squared_error: 5.4342 - rmse: 2.3311 - mean_absolute_error: 1.8447
Epoch 00010: val_loss improved from 40609.30859 to 31212.90430, saving model to /tmp/fcn.m5
4/4 [==============================] - 1s 250ms/step - loss: 5.4342 - mean_squared_error: 5.4342 - rmse: 2.3311 - mean_absolute_error: 1.8447 - val_loss: 31212.9043 - val_mean_squared_error: 31212.9043 - val_rmse: 176.6717 - val_mean_absolute_error: 175.5993 - lr: 0.0100
Epoch 11/1

Epoch 26/1000
4/4 [==============================] - ETA: 0s - loss: 1.7410 - mean_squared_error: 1.7410 - rmse: 1.3195 - mean_absolute_error: 1.0519
Epoch 00026: val_loss did not improve from 678.75598
4/4 [==============================] - 1s 228ms/step - loss: 1.7410 - mean_squared_error: 1.7410 - rmse: 1.3195 - mean_absolute_error: 1.0519 - val_loss: 1186.7461 - val_mean_squared_error: 1186.7461 - val_rmse: 34.4492 - val_mean_absolute_error: 33.8942 - lr: 0.0100
Epoch 27/1000
4/4 [==============================] - ETA: 0s - loss: 1.6216 - mean_squared_error: 1.6216 - rmse: 1.2734 - mean_absolute_error: 1.0006
Epoch 00027: val_loss did not improve from 678.75598
4/4 [==============================] - 1s 234ms/step - loss: 1.6216 - mean_squared_error: 1.6216 - rmse: 1.2734 - mean_absolute_error: 1.0006 - val_loss: 764.7416 - val_mean_squared_error: 764.7416 - val_rmse: 27.6540 - val_mean_absolute_error: 27.2050 - lr: 0.0100
Epoch 28/1000
4/4 [==============================] - ETA: 0s

4/4 [==============================] - ETA: 0s - loss: 0.7804 - mean_squared_error: 0.7804 - rmse: 0.8834 - mean_absolute_error: 0.6668
Epoch 00043: val_loss did not improve from 11.79494
4/4 [==============================] - 1s 228ms/step - loss: 0.7804 - mean_squared_error: 0.7804 - rmse: 0.8834 - mean_absolute_error: 0.6668 - val_loss: 57.7050 - val_mean_squared_error: 57.7050 - val_rmse: 7.5964 - val_mean_absolute_error: 6.8182 - lr: 0.0100
Epoch 44/1000
4/4 [==============================] - ETA: 0s - loss: 1.0523 - mean_squared_error: 1.0523 - rmse: 1.0258 - mean_absolute_error: 0.7297
Epoch 00044: val_loss did not improve from 11.79494
4/4 [==============================] - 1s 232ms/step - loss: 1.0523 - mean_squared_error: 1.0523 - rmse: 1.0258 - mean_absolute_error: 0.7297 - val_loss: 26.9686 - val_mean_squared_error: 26.9686 - val_rmse: 5.1931 - val_mean_absolute_error: 4.4780 - lr: 0.0100
Epoch 45/1000
4/4 [==============================] - ETA: 0s - loss: 0.9606 - mean_squ

Epoch 61/1000
4/4 [==============================] - ETA: 0s - loss: 0.5493 - mean_squared_error: 0.5493 - rmse: 0.7411 - mean_absolute_error: 0.5774
Epoch 00061: val_loss did not improve from 10.75844
4/4 [==============================] - 1s 227ms/step - loss: 0.5493 - mean_squared_error: 0.5493 - rmse: 0.7411 - mean_absolute_error: 0.5774 - val_loss: 20.4526 - val_mean_squared_error: 20.4526 - val_rmse: 4.5225 - val_mean_absolute_error: 3.8450 - lr: 0.0100
Epoch 62/1000
4/4 [==============================] - ETA: 0s - loss: 0.4166 - mean_squared_error: 0.4166 - rmse: 0.6454 - mean_absolute_error: 0.4595
Epoch 00062: val_loss did not improve from 10.75844
4/4 [==============================] - 1s 233ms/step - loss: 0.4166 - mean_squared_error: 0.4166 - rmse: 0.6454 - mean_absolute_error: 0.4595 - val_loss: 10.8686 - val_mean_squared_error: 10.8686 - val_rmse: 3.2968 - val_mean_absolute_error: 2.5567 - lr: 0.0100
Epoch 63/1000
4/4 [==============================] - ETA: 0s - loss: 0.5

...

Epoch 285/1000
2/2 [==============================] - ETA: 0s - loss: 0.5563 - mean_squared_error: 0.5563 - rmse: 0.7459 - mean_absolute_error: 0.5614
Epoch 00285: val_loss did not improve from 4.89283
2/2 [==============================] - 0s 109ms/step - loss: 0.5563 - mean_squared_error: 0.5563 - rmse: 0.7459 - mean_absolute_error: 0.5614 - val_loss: 5.1747 - val_mean_squared_error: 5.1747 - val_rmse: 2.2748 - val_mean_absolute_error: 1.8030 - lr: 3.9062e-05
Epoch 286/1000
2/2 [==============================] - ETA: 0s - loss: 0.9429 - mean_squared_error: 0.9429 - rmse: 0.9710 - mean_absolute_error: 0.7336
Epoch 00286: val_loss did not improve from 4.89283
2/2 [==============================] - 0s 107ms/step - loss: 0.9429 - mean_squared_error: 0.9429 - rmse: 0.9710 - mean_absolute_error: 0.7336 - val_loss: 5.2298 - val_mean_squared_error: 5.2298 - val_rmse: 2.2869 - val_mean_absolute_error: 1.8285 - lr: 3.9062e-05
Epoch 287/1000
2/2 [==============================] - ETA: 0s - loss

Epoch 303/1000
2/2 [==============================] - ETA: 0s - loss: 0.4946 - mean_squared_error: 0.4946 - rmse: 0.7033 - mean_absolute_error: 0.5482
Epoch 00303: val_loss did not improve from 4.89283
2/2 [==============================] - 0s 106ms/step - loss: 0.4946 - mean_squared_error: 0.4946 - rmse: 0.7033 - mean_absolute_error: 0.5482 - val_loss: 5.4019 - val_mean_squared_error: 5.4019 - val_rmse: 2.3242 - val_mean_absolute_error: 1.8994 - lr: 1.9531e-05
Epoch 304/1000
2/2 [==============================] - ETA: 0s - loss: 0.4739 - mean_squared_error: 0.4739 - rmse: 0.6884 - mean_absolute_error: 0.5524
Epoch 00304: val_loss did not improve from 4.89283
2/2 [==============================] - 0s 109ms/step - loss: 0.4739 - mean_squared_error: 0.4739 - rmse: 0.6884 - mean_absolute_error: 0.5524 - val_loss: 5.3678 - val_mean_squared_error: 5.3678 - val_rmse: 2.3169 - val_mean_absolute_error: 1.8890 - lr: 1.9531e-05
Epoch 305/1000
2/2 [==============================] - ETA: 0s - loss

Epoch 321/1000
2/2 [==============================] - ETA: 0s - loss: 0.4967 - mean_squared_error: 0.4967 - rmse: 0.7048 - mean_absolute_error: 0.5724
Epoch 00321: val_loss did not improve from 4.89283
2/2 [==============================] - 0s 112ms/step - loss: 0.4967 - mean_squared_error: 0.4967 - rmse: 0.7048 - mean_absolute_error: 0.5724 - val_loss: 5.2093 - val_mean_squared_error: 5.2093 - val_rmse: 2.2824 - val_mean_absolute_error: 1.8065 - lr: 9.7656e-06
Epoch 322/1000
2/2 [==============================] - ETA: 0s - loss: 0.5266 - mean_squared_error: 0.5266 - rmse: 0.7257 - mean_absolute_error: 0.5790
Epoch 00322: val_loss did not improve from 4.89283
2/2 [==============================] - 0s 104ms/step - loss: 0.5266 - mean_squared_error: 0.5266 - rmse: 0.7257 - mean_absolute_error: 0.5790 - val_loss: 5.2118 - val_mean_squared_error: 5.2118 - val_rmse: 2.2829 - val_mean_absolute_error: 1.8072 - lr: 9.7656e-06
Epoch 323/1000
2/2 [==============================] - ETA: 0s - loss

Epoch 339/1000
2/2 [==============================] - ETA: 0s - loss: 0.5134 - mean_squared_error: 0.5134 - rmse: 0.7165 - mean_absolute_error: 0.5617
Epoch 00339: val_loss did not improve from 4.89283
2/2 [==============================] - 0s 103ms/step - loss: 0.5134 - mean_squared_error: 0.5134 - rmse: 0.7165 - mean_absolute_error: 0.5617 - val_loss: 5.1394 - val_mean_squared_error: 5.1394 - val_rmse: 2.2670 - val_mean_absolute_error: 1.7953 - lr: 4.8828e-06
Epoch 340/1000
2/2 [==============================] - ETA: 0s - loss: 0.6522 - mean_squared_error: 0.6522 - rmse: 0.8076 - mean_absolute_error: 0.6288
Epoch 00340: val_loss did not improve from 4.89283
2/2 [==============================] - 0s 108ms/step - loss: 0.6522 - mean_squared_error: 0.6522 - rmse: 0.8076 - mean_absolute_error: 0.6288 - val_loss: 5.1321 - val_mean_squared_error: 5.1321 - val_rmse: 2.2654 - val_mean_absolute_error: 1.7909 - lr: 4.8828e-06
Epoch 341/1000
2/2 [==============================] - ETA: 0s - loss

Epoch 357/1000
2/2 [==============================] - ETA: 0s - loss: 0.5830 - mean_squared_error: 0.5830 - rmse: 0.7636 - mean_absolute_error: 0.6115
Epoch 00357: val_loss did not improve from 4.89283
2/2 [==============================] - 0s 102ms/step - loss: 0.5830 - mean_squared_error: 0.5830 - rmse: 0.7636 - mean_absolute_error: 0.6115 - val_loss: 5.1086 - val_mean_squared_error: 5.1086 - val_rmse: 2.2602 - val_mean_absolute_error: 1.7796 - lr: 2.4414e-06
Epoch 358/1000
2/2 [==============================] - ETA: 0s - loss: 0.7787 - mean_squared_error: 0.7787 - rmse: 0.8824 - mean_absolute_error: 0.6552
Epoch 00358: val_loss did not improve from 4.89283
2/2 [==============================] - 0s 106ms/step - loss: 0.7787 - mean_squared_error: 0.7787 - rmse: 0.8824 - mean_absolute_error: 0.6552 - val_loss: 5.1102 - val_mean_squared_error: 5.1102 - val_rmse: 2.2606 - val_mean_absolute_error: 1.7798 - lr: 2.4414e-06
Epoch 359/1000
2/2 [==============================] - ETA: 0s - loss

In [72]:
df_fcn = (df_fcn1.merge(df_fcn2, how='outer')).merge(df_fcn3, how='outer')

In [157]:
def to_arr(dict):
    lists = dict.items()
    x,y = zip(*lists)
    arr = np.array(y)
    return arr

In [171]:
to_arr(results_1_3)[0,:]          

array([11.16009808,  3.34067321,  2.61717224])

In [75]:
df_lenet


,Model,Train_site,Test_site,Metric,Value
0,cnn_lenet,0,0,MSE,13.840021
1,cnn_lenet,0,0,RMSE,3.720218
2,cnn_lenet,0,0,MAE,2.866733
3,cnn_lenet,0,0,coef,0.312581
4,cnn_lenet,0,3,MSE,15.715021
5,cnn_lenet,0,3,RMSE,3.964218
6,cnn_lenet,0,3,MAE,2.945293
7,cnn_lenet,0,3,coef,0.245997
8,cnn_lenet,0,4,MSE,15.704589
9,cnn_lenet,0,4,RMSE,3.962902
